# Corpus (STAC) Uploading

In [ ]:
# === Step 1: Upload the Excel file ===
from google.colab import files
import pandas as pd
import json

# Prompt file upload
uploaded = files.upload()

# Automatically get the uploaded filename
xlsx_filename = next(iter(uploaded))

# Load Excel file
df = pd.read_csv(xlsx_filename)

# Rename columns for consistency
# Eename 'dialogue_num' → 'Dialogue_ID' and 'text' → 'EDU_text'
df = df.rename(columns={
    "dialogue_num": "Dialogue_ID",
    "text": "EDU_text"
})

# Check necessary columns exist
assert 'Dialogue_ID' in df.columns, "Missing column: 'Dialogue_ID'"
assert 'EDU_text' in df.columns, "Missing column: 'EDU_text'"

# Group EDUs by dialogue
grouped = df.groupby("Dialogue_ID")["EDU_text"].apply(list)

# Filter dialogues with >2 EDUs only
filtered_dialogues = [
    {"dialogue_id": str(dialogue_id), "edus": edus}
    for dialogue_id, edus in grouped.items()
    if 2 <= len(edus)
]

# Save to JSON Lines format
output_path = "/content/preprocessed_dialogues.jsonl"
with open(output_path, "w", encoding="utf-8") as f:
    for dialogue in filtered_dialogues:
        json.dump(dialogue, f, ensure_ascii=False)
        f.write("\n")

print(f"Preprocessing complete. {len(filtered_dialogues)} dialogues saved to: {output_path}")

Saving incoming_base_spect.csv to incoming_base_spect.csv
Preprocessing complete. 1101 dialogues saved to: /content/preprocessed_dialogues.jsonl


# BART Sentence Ordering Preprocessing

In [ ]:
import json
import random
import pandas as pd
from collections import Counter
from pathlib import Path
from google.colab import files

# Load CSV
df = pd.read_csv("incoming_base_spect.csv")

# Group by dialogue (exclude rows with '0' relation)
grouped = df.groupby(['doc', 'dialogue_num'])

processed_dialogues = []

# Preprocess each dialogue
for (doc_id, dlg_num), group in grouped:
    group = group.dropna(subset=['text', 'relation_type'])
    edus = [e.strip() for e in group['text'].tolist() if e.strip() and e.strip() != "0"]
    relations = [r.strip() for r in group['relation_type'].tolist() if r.strip() and r.strip() != "0"]

    if len(edus) < 3:
        continue

    shuffled = edus.copy()
    for _ in range(10):
        random.shuffle(shuffled)
        if shuffled != edus:
            break
    else:
        continue

    rel_counts = Counter(relations)
    if not rel_counts:
        continue

    max_count = max(rel_counts.values())
    dominant_relations = [rel for rel, count in rel_counts.items() if count == max_count]
    if len(dominant_relations) != 1:
        continue

    dominant_relation = dominant_relations[0]
    deduplicated_edus = list(dict.fromkeys(edus))

    processed_dialogues.append({
        "dialogue_id": f"{doc_id}_{dlg_num}",
        "source": " ||| ".join(shuffled).strip(),
        "target": " ||| ".join(deduplicated_edus).strip(),
        "dominant_relation": dominant_relation
    })

# Save full dataset
with open("bart_sentence_ordering.jsonl", "w", encoding="utf-8") as f:
    for item in processed_dialogues:
        json.dump(item, f, ensure_ascii=False)
        f.write("\n")
files.download("bart_sentence_ordering.jsonl")

# Split into five folds (train/test only)
def make_5_fold_train_test_splits(data, seed=42):
    random.seed(seed)
    random.shuffle(data)
    k = 5
    fold_size = len(data) // k
    remainder = len(data) % k

    fold_sizes = [fold_size] * k
    for i in range(remainder):
        fold_sizes[i] += 1

    folds = []
    idx = 0
    for size in fold_sizes:
        folds.append(data[idx:idx + size])
        idx += size

    return folds

# Save all five train/test folds
def save_5_fold_splits(folds):
    for i in range(5):
        test = folds[i]
        train = [item for j, fold in enumerate(folds) if j != i for item in fold]

        for name, split_data in zip(["train", "test"], [train, test]):
            filename = f"bart_ordering_fold{i+1}_{name}.jsonl"
            with open(filename, "w", encoding="utf-8") as f:
                for item in split_data:
                    json.dump(item, f, ensure_ascii=False)
                    f.write("\n")
            print(f"Fold {i+1} | {name}: {len(split_data)} examples → {filename}")
            files.download(filename)

# Run everything
folds = make_5_fold_train_test_splits(processed_dialogues)
save_5_fold_splits(folds)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Fold 1 | train: 572 examples → bart_ordering_fold1_train.jsonl


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Fold 5 | test: 143 examples → bart_ordering_fold5_test.jsonl


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# BART Sentence Ordering Fine-tuning

In [ ]:
# Install Required Packages
!pip install transformers datasets scipy --quiet

# Imports
import json
import gc
import torch
from torch.utils.data import Dataset
from transformers import (
    BartTokenizerFast, BartForConditionalGeneration,
    Trainer, TrainingArguments, EarlyStoppingCallback
)
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Load Tokenizer and Add Custom Token
tokenizer = BartTokenizerFast.from_pretrained("facebook/bart-base")
tokenizer.add_tokens(["[REL]"])

# Dataset Class
class BartOrderingDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        source = item["source"]
        target = item["target"]

        inputs = self.tokenizer(
            source,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        targets = self.tokenizer(
            target,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        labels = targets.input_ids.squeeze()
        labels[labels == tokenizer.pad_token_id] = -100

        return {
            "input_ids": inputs.input_ids.squeeze(),
            "attention_mask": inputs.attention_mask.squeeze(),
            "labels": labels
        }

# Helper to Load JSONL
def load_jsonl(path):
    with open(path, "r", encoding="utf-8") as f:
        return [json.loads(line.strip()) for line in f]

# 5-Fold Training Loop
for i in range(1, 6):
    print(f"\n Training Fold {i}...")

    torch.cuda.empty_cache()
    gc.collect()

    train_data = load_jsonl(f"bart_ordering_fold{i}_train.jsonl")
    test_data = load_jsonl(f"bart_ordering_fold{i}_test.jsonl")

    train_dataset = BartOrderingDataset(train_data, tokenizer)
    test_dataset = BartOrderingDataset(test_data, tokenizer)

    # Load smaller model
    model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")
    model.resize_token_embeddings(len(tokenizer))

    # Training arguments
    training_args = TrainingArguments(
        output_dir=f"bart_ordering_model_fold{i}",
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,
        gradient_accumulation_steps=2,
        num_train_epochs=20,
        eval_strategy="epoch",
        save_strategy="epoch",
        save_total_limit=1,
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False,
        weight_decay=0.01,
        logging_dir=f"logs_fold{i}",
        logging_steps=10,
        fp16=True,
        label_smoothing_factor=0.1,
        report_to="none"
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    )

    trainer.train()

    model.save_pretrained(f"bart_ordering_model_fold{i}")
    tokenizer.save_pretrained(f"bart_ordering_model_fold{i}")

    del model
    del trainer
    torch.cuda.empty_cache()
    gc.collect()

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]


 Training Fold 1...


...

Epoch,Training Loss,Validation Loss
1,5.269100,5.307625
2,5.337400,5.160767
3,5.025500,5.056846
4,5.025800,5.052581
5,4.736900,5.052624
6,4.711700,5.082236
7,4.680300,5.130821


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3922: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].



 Training Fold 5...


Epoch,Training Loss,Validation Loss
1,5.443000,5.201966
2,5.217700,5.062531
3,5.056400,4.991839
4,4.999200,4.931699
5,4.702400,4.947175
6,4.655700,4.916999
7,4.571000,4.930861
8,4.718100,4.962973
9,4.489700,5.016042


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3922: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


# Fine-tuned BART Sentence Ordering Evaluation

In [ ]:
import json
from collections import defaultdict
from scipy.stats import spearmanr
from difflib import SequenceMatcher
import math
import torch
from transformers import BartTokenizerFast, BartForConditionalGeneration

def load_jsonl(path):
    with open(path, 'r', encoding='utf-8') as f:
        return [json.loads(line.strip()) for line in f]

def clean_edus_list(edus):
    return [e.strip() for e in edus if e.strip() and e.strip() != "0"]

def align_pred_to_gold(pred_edus, gold_edus):
    aligned = []
    used = set()
    for pred in pred_edus:
        best_match = None
        best_score = 0.0
        for gold in gold_edus:
            if gold in used:
                continue
            score = SequenceMatcher(None, pred, gold).ratio()
            if score > best_score:
                best_score = score
                best_match = gold
        if best_match:
            aligned.append(best_match)
            used.add(best_match)
    for g in gold_edus:
        if g not in aligned:
            aligned.append(g)
    return aligned[:len(gold_edus)]

# Settings
max_len = 768
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Global Statistics
global_relation_scores = defaultdict(list)
global_relation_counts = defaultdict(int)
all_rhos = []

# Run Evaluation for Each Fold
for i in range(1, 6):
    print(f"\n================ Fold {i} Evaluation ================\n")

    tokenizer = BartTokenizerFast.from_pretrained(f"bart_ordering_model_fold{i}")
    model = BartForConditionalGeneration.from_pretrained(f"bart_ordering_model_fold{i}")
    model.to(device)
    model.eval()

    test_data = load_jsonl(f"bart_ordering_fold{i}_test.jsonl")
    relation_scores = defaultdict(list)
    total_scores = []
    skipped = 0

    for example in test_data:
        gold_edus = clean_edus_list(example["target"].split("|||"))
        if len(gold_edus) < 3:
            skipped += 1
            continue

        inputs = tokenizer(
            example["source"],
            return_tensors="pt",
            truncation=True,
            padding="max_length",
            max_length=max_len
        )
        inputs = {k: v.to(device) for k, v in inputs.items()}

        with torch.no_grad():
            output_ids = model.generate(
                input_ids=inputs["input_ids"],
                attention_mask=inputs["attention_mask"],
                max_length=max_len,
                num_beams=4
            )
        pred_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
        pred_edus = clean_edus_list(pred_text.split("|||"))
        aligned_pred = align_pred_to_gold(pred_edus, gold_edus)

        if len(aligned_pred) != len(gold_edus):
            skipped += 1
            continue

        try:
            pred_ranks = [gold_edus.index(e) for e in aligned_pred]
            gold_ranks = list(range(len(gold_edus)))
            rho, _ = spearmanr(gold_ranks, pred_ranks)
            if math.isnan(rho):
                skipped += 1
                continue
        except:
            skipped += 1
            continue

        rel = example.get("dominant_relation", "Unknown")
        relation_scores[rel].append(rho)
        total_scores.append(rho)

        global_relation_scores[rel].append(rho)
        global_relation_counts[rel] += 1
        all_rhos.append(rho)

        # Print every example
        print("\n Dialogue Example:")
        print(f" Relation: {rel}")
        print(f" Spearman’s ρ: {rho:.4f}")
        print("\n Predicted Sequence:")
        for line in aligned_pred:
            print(f"  {line}")
        print("\n Gold Sequence:")
        for line in gold_edus:
            print(f"  {line}")

    print("\n--- Average Spearman’s ρ by Relation Type ---")
    for rel, scores in relation_scores.items():
        mean_rho = sum(scores) / len(scores)
        print(f"{rel:<20} n={len(scores):>3}  |  Mean ρ: {mean_rho:.4f}")

    if total_scores:
        overall_avg = sum(total_scores) / len(total_scores)
        print(f"\nMean Spearman’s ρ (all examples): {overall_avg:.4f} (n={len(total_scores)})")
    else:
        print("No valid samples.")

    print(f"\n Skipped examples: {skipped}")

# Global Summary
print("\n================ Overall Evaluation Summary ================\n")
print("--- Average Spearman’s ρ by Relation Type (All Folds) ---")
for rel, scores in global_relation_scores.items():
    count = global_relation_counts[rel]
    mean_rho = sum(scores) / len(scores)
    print(f"{rel:<25} n={count:<4} | Mean ρ: {mean_rho:.4f}")

if all_rhos:
    unweighted_avg_all = sum(all_rhos) / len(all_rhos)
    print(f"\nMean Spearman’s ρ (all examples): {unweighted_avg_all:.4f}")
else:
    print("No valid overall statistics.")

Streaming output truncated to the last 5000 lines.
  I'll give wheat...
  Give me clay
  and we might have a deal.
  sorry
  how about 2 wheat.. for 1 wood?
  Sorry...
  how about 1 ore and 1 wheat for wood
  ok...
  Still can't do it..
  you want 1 wood?
  i did..
  but just traded with bank...
  haha
  yer... expensive banks.. just like real life..
  indeed it is

 Dialogue Example:
 Relation: Continuation
 Spearman’s ρ: -0.2000

 Predicted Sequence:
  skinny, got sheep?
  No worries
  Also, neither of you can build a settlement without sheep...
  Nope
  sorry

 Gold Sequence:
  sorry
  No worries
  skinny, got sheep?
  Nope
  Also, neither of you can build a settlement without sheep...

 Dialogue Example:
 Relation: Question_answer_pair
 Spearman’s ρ: 0.4759

 Predicted Sequence:
  Yeah,
  so it's OK.
  I'm good for a little bit longer.
  Ah. Fun I take it?
  So, does this generate a random Catan board?
  I guess if person number 3 doesn't show up in 5 or 10 minutes
  Ahh! I see.
  

## Significance utilities


In [ ]:
# Significance utilities (overall + by relation)
import os, math, numpy as np, pandas as pd
from scipy.stats import wilcoxon

def _merge_baseline_finetuned(baseline_csv: str, finetuned_csv: str) -> pd.DataFrame:
    b = pd.read_csv(baseline_csv)
    f = pd.read_csv(finetuned_csv)
    for req in ["dialogue_id", "relation_type", "metric_value"]:
        if req not in b.columns or req not in f.columns:
            raise ValueError(f"Missing '{req}' in baseline or finetuned CSV.")
    df = pd.merge(b, f, on="dialogue_id", suffixes=("_baseline", "_finetuned"))
    # Prefer the finetuned relation label if both exist
    df["relation_type"] = df.get("relation_type_finetuned", df.get("relation_type_baseline", "Unknown"))
    df = df[["dialogue_id", "relation_type", "metric_value_baseline", "metric_value_finetuned"]]
    df = df.dropna(subset=["metric_value_baseline", "metric_value_finetuned"]).copy()
    if len(df) == 0:
        raise ValueError("No overlapping dialogues (or all NA) after merge.")
    return df

def _bootstrap_mean_diff_CI(x, y, n_boot=10000, seed=42, alpha=0.05):
    rng = np.random.default_rng(seed)
    diffs = np.asarray(x) - np.asarray(y)
    n = len(diffs)
    idx = rng.integers(0, n, size=(n_boot, n))
    boot_means = diffs[idx].mean(axis=1)
    low = float(np.quantile(boot_means, alpha/2))
    high = float(np.quantile(boot_means, 1 - alpha/2))
    return low, high

def run_significance_on_csvs(baseline_csv: str, finetuned_csv: str, *,
                             out_prefix: str,
                             min_n_per_rel: int = 8,
                             metric_name: str = "ρ"):
    """
    baseline_csv / finetuned_csv columns: dialogue_id, relation_type, metric_value
    Saves: {out_prefix}_significance_overall.csv, {out_prefix}_significance_by_relation.csv
    Returns (overall_df, by_relation_df)
    """
    df = _merge_baseline_finetuned(baseline_csv, finetuned_csv)

    x = df["metric_value_finetuned"].to_numpy()
    y = df["metric_value_baseline"].to_numpy()

    # Overall paired Wilcoxon + one-sided chance test (finetuned > 0)
    w_stat, w_p = wilcoxon(x, y, zero_method="wilcox", alternative="two-sided")
    w_stat_chance, w_p_chance = wilcoxon(x, np.zeros_like(x), zero_method="wilcox", alternative="greater")
    ci_low, ci_high = _bootstrap_mean_diff_CI(x, y)

    overall = pd.DataFrame([{
        "n_dialogues": int(len(df)),
        f"baseline_mean_{metric_name}": float(y.mean()),
        f"finetuned_mean_{metric_name}": float(x.mean()),
        "mean_diff": float(x.mean() - y.mean()),
        "boot_CI_low": ci_low,
        "boot_CI_high": ci_high,
        "wilcoxon_stat": float(w_stat),
        "wilcoxon_p": float(w_p),
        f"chance_test_p(finetuned>{0})": float(w_p_chance),
    }])

    rows = []
    for rel, grp in df.groupby("relation_type"):
        n = len(grp)
        base = grp["metric_value_baseline"].to_numpy()
        fine = grp["metric_value_finetuned"].to_numpy()
        mean_b = float(base.mean())
        mean_f = float(fine.mean())
        diff = float(mean_f - mean_b)
        if n >= min_n_per_rel:
            try:
                ws, wp = wilcoxon(fine, base, zero_method="wilcox", alternative="two-sided")
            except ValueError:
                ws, wp = math.nan, math.nan
            try:
                ws_c, wp_c = wilcoxon(fine, np.zeros_like(fine), zero_method="wilcox", alternative="greater")
            except ValueError:
                ws_c, wp_c = math.nan, math.nan
        else:
            ws = wp = ws_c = wp_c = math.nan
        rows.append({
            "relation_type": rel,
            "n": int(n),
            f"baseline_mean_{metric_name}": mean_b,
            f"finetuned_mean_{metric_name}": mean_f,
            "mean_diff": diff,
            "wilcoxon_p(baseline_vs_finetuned)": "" if (isinstance(wp, float) and math.isnan(wp)) else wp,
            f"chance_test_p(finetuned>{0})": "" if (isinstance(wp_c, float) and math.isnan(wp_c)) else wp_c,
        })
    byrel = pd.DataFrame(rows).sort_values(["n", "relation_type"], ascending=[False, True])

    os.makedirs(os.path.dirname(out_prefix), exist_ok=True)
    overall_path = f"{out_prefix}_significance_overall.csv"
    byrel_path   = f"{out_prefix}_significance_by_relation.csv"
    overall.to_csv(overall_path, index=False)
    byrel.to_csv(byrel_path, index=False)
    print("Saved:")
    print("-", overall_path)
    print("-", byrel_path)
    return overall, byrel

def print_sig_paragraph(overall_df: pd.DataFrame, *, label: str, metric_name: str = "ρ"):
    r = overall_df.iloc[0]
    b = r[f"baseline_mean_{metric_name}"]; f = r[f"finetuned_mean_{metric_name}"]
    p = r["wilcoxon_p"]; lo = r["boot_CI_low"]; hi = r["boot_CI_high"]
    pchance = r.get(f"chance_test_p(finetuned>{0})", None)
    msg = (f"Significance ({label}). The improvement from baseline ({metric_name} = {b:.4f}) "
           f"to fine-tuned ({metric_name} = {f:.4f}) was statistically significant "
           f"(Wilcoxon signed-rank, p = {p:.3g}; 95% CI for mean difference: [{lo:.3f}, {hi:.3f}]).")
    if pchance is not None:
        msg += f" Fine-tuned performance was also significantly above chance ({metric_name} > 0; p = {pchance:.3g})."
    print(msg)

## Export finetuned per-dialogue CSV

In [ ]:
# Export FINETUNED per-dialogue CSV for BART ordering from processed_dialogues + all_rhos
import os, pandas as pd, numpy as np
os.makedirs("/content/exports", exist_ok=True)

def _extract_id_rel(d):
    did = None; rel = None
    if isinstance(d, dict):
        for k in ("dialogue_id","dialogueId","doc","id","name"):
            if k in d:
                did = d[k]; break
        for k in ("dominant_relation","dominantRelation","relation_type","dominant_rel","dominant"):
            if k in d:
                rel = d[k]; break
    if did is None:
        pass
    return (str(did) if did is not None else None,
            str(rel) if rel is not None else "Unknown")

L = min(len(processed_dialogues), len(all_rhos))
rows = []
for i in range(L):
    d = processed_dialogues[i]
    did, rel = _extract_id_rel(d)
    if did is None:
        did = f"dlg_{i:04d}"
    rho = float(all_rhos[i])
    rows.append({"dialogue_id": did, "relation_type": rel, "metric_value": rho})

df_fin = pd.DataFrame(rows)
fin_path = "/content/exports/bart_ordering_finetuned.csv"
df_fin.to_csv(fin_path, index=False)
print("Saved →", fin_path, "rows:", len(df_fin))
display(df_fin.head(5))

Saved → /content/exports/bart_ordering_finetuned.csv rows: 712


,dialogue_id,relation_type,metric_value
0,s2_leagueM_game4_1056.0,Question_answer_pair,0.942857
1,s1_league1_game2_281.0,Question_answer_pair,1.000000
2,s2_league3_game1_666.0,Comment,0.600000
3,s1_league1_game5_383.0,Comment,1.000000
4,s2_league5_game4_936.0,Question_answer_pair,-0.371429


# BART Sentence Ordering Baseline Evaluation

In [ ]:
# Load the base checkpoint
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch, gc

torch.cuda.empty_cache(); gc.collect()

BASE_CKPT = "facebook/bart-base"
tokenizer_base = AutoTokenizer.from_pretrained(BASE_CKPT)
model_base = AutoModelForSeq2SeqLM.from_pretrained(BASE_CKPT)
model_base.eval()
_ = model_base.to("cuda" if torch.cuda.is_available() else "cpu")
print("Loaded baseline model:", BASE_CKPT)


Loaded baseline model: facebook/bart-base


In [ ]:
import json
import math
import os
from collections import defaultdict
from scipy.stats import spearmanr
from difflib import SequenceMatcher

# Load Test Data
def load_jsonl(path):
    with open(path, 'r', encoding='utf-8') as f:
        return [json.loads(line.strip()) for line in f]

def rank_order(edus):
    return [edus.index(e) for e in edus]

# Fuzzy alignment
def align_pred_to_gold_fuzzy(pred_edus, gold_edus):
    """
    Map each predicted EDU to the most similar unmatched gold EDU using
    SequenceMatcher.ratio(), then append any remaining gold EDUs to preserve length.
    """
    used = set()
    aligned = []
    for p in pred_edus:
        best_j = -1
        best_s = -1.0
        for j, g in enumerate(gold_edus):
            if j in used:
                continue
            s = SequenceMatcher(None, p, g).ratio()
            if s > best_s:
                best_s = s
                best_j = j
        if best_j != -1:
            used.add(best_j)
            aligned.append(gold_edus[best_j])
    # append any gold EDUs that weren't matched to keep lengths equal
    for j, g in enumerate(gold_edus):
        if j not in used:
            aligned.append(g)
    return aligned[:len(gold_edus)]

# Evaluation Loop
print("\n--- Heuristic Baseline Reordering: Spearman Evaluation (All Folds) ---")
relation_scores = defaultdict(list)
total_scores = []
skipped = 0

all_rhos_baseline = []
relations_baseline = []

for i in range(1, 6):
    print(f"\n========== Evaluating Fold {i} ==========\n")
    test_data = load_jsonl(f"bart_ordering_fold{i}_test.jsonl")

    for idx, example in enumerate(test_data):
        gold_edus = [e.strip() for e in example["target"].split("|||") if e.strip() and e.strip() != "0"]
        shuffled_edus = [e.strip() for e in example["source"].split("|||") if e.strip() and e.strip() != "0"]
        relation = example.get("dominant_relation", "Unknown")

        if len(gold_edus) < 3 or len(shuffled_edus) < 3:
            skipped += 1
            continue

        # Heuristic baseline: simple lexical sort + fuzzy alignment to gold
        heuristic_ordered = sorted(shuffled_edus)
        aligned_pred = align_pred_to_gold_fuzzy(heuristic_ordered, gold_edus)

        try:
            pred_ranks = [gold_edus.index(e) for e in aligned_pred]
            gold_ranks = list(range(len(gold_edus)))
            rho, _ = spearmanr(gold_ranks, pred_ranks)
        except Exception:
            skipped += 1
            continue

        if math.isnan(rho):
            skipped += 1
            continue

        relation_scores[relation].append(rho)
        total_scores.append(rho)

        all_rhos_baseline.append(float(rho))
        relations_baseline.append(relation)

        # Print example
        print(f"\n Fold {i}, Example {idx}")
        print(f" Relation Type: {relation}")
        print(f" Spearman’s ρ: {rho:.4f}")
        print("\n Heuristic Prediction:")
        for edu in aligned_pred:
            print("  ", edu)
        print("\n Gold Sequence:")
        for edu in gold_edus:
            print("  ", edu)

# Report Summary by Relation
print("\n--- Average Spearman’s ρ by Relation Type (All Folds) ---")
for rel, scores in relation_scores.items():
    mean_rho = sum(scores) / len(scores)
    print(f"{rel:<25} n={len(scores):>3}  |  Mean ρ: {mean_rho:.4f}")

# Final Averages
if total_scores:
    overall_avg = sum(total_scores) / len(total_scores)
    print(f"\nMean Spearman’s ρ (all examples): {overall_avg:.4f} (n={len(total_scores)})")
else:
    print("\n No valid Spearman scores computed.")

print(f"\nTotal evaluated: {len(total_scores)}, Skipped: {skipped}")
print(f"Collected per-dialogue ρs for baseline: {len(all_rhos_baseline)}")

Streaming output truncated to the last 5000 lines.
   eh?
   how about 2 wheat.. for 1 wood?
   how about 1 ore and 1 wheat for wood
   you want 1 wood?
   ooo..
   ok...
   indeed it is
   misprpressed to end my go!
   sorry
   wood or ore
   yer... expensive banks.. just like real life..
   sheep

 Gold Sequence:
   What do you need Shawnus
   sheep
   What can you give?
   wood or ore
   damn
   eh?
   ooo..
   7..
   Wood would be nice
   Ho... no..
   misprpressed to end my go!
   anyone want to give me wood..
   I'll give wheat...
   Give me clay
   and we might have a deal.
   sorry
   how about 2 wheat.. for 1 wood?
   Sorry...
   how about 1 ore and 1 wheat for wood
   ok...
   Still can't do it..
   you want 1 wood?
   i did..
   but just traded with bank...
   haha
   yer... expensive banks.. just like real life..
   indeed it is

 Fold 4, Example 130
 Relation Type: Continuation
 Spearman’s ρ: -0.7000

 Heuristic Prediction:
   Also, neither of you can build a settlement wi

## Export baseline CSV

In [ ]:
# Export BASELINE per-dialogue CSV for BART ordering
import os, pandas as pd

BASELINE_OUT = "/content/exports/bart_ordering_baseline.csv"
FINETUNED_IN = "/content/exports/bart_ordering_finetuned.csv"
os.makedirs("/content/exports", exist_ok=True)

def _extract_id_rel(d):
    did = None; rel = None
    if isinstance(d, dict):
        for k in ("dialogue_id","dialogueId","doc","id","name"):
            if k in d:
                did = d[k]; break
        for k in ("dominant_relation","dominantRelation","relation_type","dominant_rel","dominant"):
            if k in d:
                rel = d[k]; break
    return (str(did) if did is not None else None,
            str(rel) if rel is not None else "Unknown")

assert 'all_rhos_baseline' in globals() and len(all_rhos_baseline) > 0, \
    "Baseline rhos not found. Make sure the baseline eval cell ran and filled 'all_rhos_baseline'."

# Preferred path: reuse finetuned CSV to guarantee perfect pairing by dialogue_id
if os.path.exists(FINETUNED_IN):
    fin = pd.read_csv(FINETUNED_IN)
    N = min(len(fin), len(all_rhos_baseline))
    df_base = pd.DataFrame({
        "dialogue_id":   fin["dialogue_id"].iloc[:N].astype(str).tolist(),
        "relation_type": fin["relation_type"].iloc[:N].astype(str).tolist(),
        "metric_value":  [float(x) for x in all_rhos_baseline[:N]],
    })
else:
    assert 'processed_dialogues' in globals() and len(processed_dialogues) > 0, \
        "Neither finetuned CSV nor processed_dialogues available to form dialogue IDs."
    N = min(len(processed_dialogues), len(all_rhos_baseline))
    rows = []
    for i in range(N):
        did, rel = _extract_id_rel(processed_dialogues[i])
        if did is None:
            did = f"dlg_{i:04d}"
        rows.append({
            "dialogue_id": did,
            "relation_type": rel,
            "metric_value": float(all_rhos_baseline[i]),
        })
    df_base = pd.DataFrame(rows)

df_base.to_csv(BASELINE_OUT, index=False)
print(f"Saved → {BASELINE_OUT}  rows: {len(df_base)}")
display(df_base.head(5))

Saved → /content/exports/bart_ordering_baseline.csv  rows: 712


,dialogue_id,relation_type,metric_value
0,s2_leagueM_game4_1056.0,Question_answer_pair,-0.657143
1,s1_league1_game2_281.0,Question_answer_pair,0.500000
2,s2_league3_game1_666.0,Comment,-0.314286
3,s1_league1_game5_383.0,Comment,-0.500000
4,s2_league5_game4_936.0,Question_answer_pair,-0.142857


## BART sentence ordering (significance)

In [ ]:
# BART sentence ordering — significance (paired Wilcoxon + CI + by-relation)
import os, math, numpy as np, pandas as pd
from scipy.stats import wilcoxon

BASELINE = "/content/exports/bart_ordering_baseline.csv"
FINETUNED = "/content/exports/bart_ordering_finetuned.csv"
OUT_PREFIX = "/content/exports/bart_ordering"
MIN_N = 8
METRIC_NAME = "ρ"

def _bootstrap_mean_diff_CI(x, y, n_boot=10000, seed=42, alpha=0.05):
    rng = np.random.default_rng(seed)
    diffs = np.asarray(x) - np.asarray(y)
    n = len(diffs)
    idx = rng.integers(0, n, size=(n_boot, n))
    boot_means = diffs[idx].mean(axis=1)
    return float(np.quantile(boot_means, alpha/2)), float(np.quantile(boot_means, 1 - alpha/2))

# Load & merge
assert os.path.exists(BASELINE), f"Missing: {BASELINE}"
assert os.path.exists(FINETUNED), f"Missing: {FINETUNED}"
b = pd.read_csv(BASELINE)
f = pd.read_csv(FINETUNED)
for req in ["dialogue_id","relation_type","metric_value"]:
    assert req in b.columns and req in f.columns, f"Missing '{req}' in one of the CSVs."

df = pd.merge(b, f, on="dialogue_id", suffixes=("_baseline","_finetuned"))
df["relation_type"] = df.get("relation_type_finetuned", df.get("relation_type_baseline","Unknown"))
df = df[["dialogue_id","relation_type","metric_value_baseline","metric_value_finetuned"]].dropna()
assert len(df) > 0, "No overlapping dialogues after merge."

# Guard: warn if arrays are identical (Wilcoxon becomes undefined)
if np.allclose(df["metric_value_baseline"], df["metric_value_finetuned"]):
    raise ValueError("Baseline and finetuned arrays are identical — rerun a true baseline and re-export.")

# Overall tests
x = df["metric_value_finetuned"].to_numpy()
y = df["metric_value_baseline"].to_numpy()
w_stat, w_p = wilcoxon(x, y, zero_method="wilcox", alternative="two-sided")
w_stat_ch, w_p_ch = wilcoxon(x, np.zeros_like(x), zero_method="wilcox", alternative="greater")
ci_lo, ci_hi = _bootstrap_mean_diff_CI(x, y)

overall = pd.DataFrame([{
    "n_dialogues": len(df),
    f"baseline_mean_{METRIC_NAME}": float(y.mean()),
    f"finetuned_mean_{METRIC_NAME}": float(x.mean()),
    "mean_diff": float(x.mean() - y.mean()),
    "boot_CI_low": ci_lo, "boot_CI_high": ci_hi,
    "wilcoxon_stat": float(w_stat), "wilcoxon_p": float(w_p),
    f"chance_test_p(finetuned>{0})": float(w_p_ch),
}])

# By-relation tests
rows = []
for rel, grp in df.groupby("relation_type"):
    n = len(grp)
    base = grp["metric_value_baseline"].to_numpy()
    fine = grp["metric_value_finetuned"].to_numpy()
    mean_b, mean_f = float(base.mean()), float(fine.mean())
    diff = float(mean_f - mean_b)
    if n >= MIN_N:
        ws, wp = wilcoxon(fine, base, zero_method="wilcox", alternative="two-sided")
        ws_c, wp_c = wilcoxon(fine, np.zeros_like(fine), zero_method="wilcox", alternative="greater")
        p_pair, p_chance = float(wp), float(wp_c)
    else:
        p_pair = p_chance = ""
    rows.append({
        "relation_type": rel, "n": int(n),
        f"baseline_mean_{METRIC_NAME}": mean_b,
        f"finetuned_mean_{METRIC_NAME}": mean_f,
        "mean_diff": diff,
        "wilcoxon_p(baseline_vs_finetuned)": p_pair,
        f"chance_test_p(finetuned>{0})": p_chance
    })
byrel = pd.DataFrame(rows).sort_values(["n","relation_type"], ascending=[False, True])

# Save + display
os.makedirs(os.path.dirname(OUT_PREFIX), exist_ok=True)
overall.to_csv(f"{OUT_PREFIX}_significance_overall.csv", index=False)
byrel.to_csv(f"{OUT_PREFIX}_significance_by_relation.csv", index=False)
display(overall); display(byrel)

r = overall.iloc[0]
print(
  f"Significance (BART sentence ordering). The improvement from baseline ({METRIC_NAME} = {r[f'baseline_mean_{METRIC_NAME}']:.4f}) "
  f"to fine-tuned ({METRIC_NAME} = {r[f'finetuned_mean_{METRIC_NAME}']:.4f}) was statistically significant "
  f"(Wilcoxon signed-rank, p = {r['wilcoxon_p']:.3g}; 95% CI for mean difference: "
  f"[{r['boot_CI_low']:.3f}, {r['boot_CI_high']:.3f}]). Fine-tuned performance was also significantly above chance "
  f"({METRIC_NAME} > 0; p = {r[f'chance_test_p(finetuned>{0})']:.3g})."
)

,n_dialogues,baseline_mean_ρ,finetuned_mean_ρ,mean_diff,boot_CI_low,boot_CI_high,wilcoxon_stat,wilcoxon_p,chance_test_p(finetuned>0)
0,712,0.130054,0.4727,0.342647,0.308549,0.377299,31815.5,4.152727e-60,1.529912e-101


,relation_type,n,baseline_mean_ρ,finetuned_mean_ρ,mean_diff,wilcoxon_p(baseline_vs_finetuned),chance_test_p(finetuned>0)
11,Question_answer_pair,429,0.130213,0.497624,0.367412,0.0,0.0
3,Comment,162,0.131974,0.449541,0.317567,0.0,0.0
0,Acknowledgement,61,0.089812,0.433749,0.343937,0.000002,0.0
4,Continuation,28,0.224762,0.384251,0.159490,0.012894,0.000037
7,Elaboration,12,0.117893,0.394479,0.276587,0.109863,0.001953
8,Explanation,4,-0.061298,0.146224,0.207522,,
10,Q_Elab,4,-0.149420,0.478281,0.627701,,
12,Result,4,0.451694,0.503984,0.052289,,
5,Contrast,3,-0.011586,0.486246,0.497832,,
1,Background,2,0.070279,0.527451,0.457172,,


Significance (BART sentence ordering). The improvement from baseline (ρ = 0.1301) to fine-tuned (ρ = 0.4727) was statistically significant (Wilcoxon signed-rank, p = 4.15e-60; 95% CI for mean difference: [0.309, 0.377]). Fine-tuned performance was also significantly above chance (ρ > 0; p = 1.53e-101).


# RoBERTa Sentence Ordering Preprocessing

In [ ]:
import pandas as pd
import json
import random
import os
from tqdm import tqdm
from collections import Counter
from sklearn.model_selection import train_test_split
from google.colab import files

# Config
input_path = "/content/incoming_base_spect.csv"
output_prefix = "/content/roberta_rerank"
allowed_relations = {"Continuation", "Elaboration", "Question_answer_pair", "Explanation", "Comment"}
num_negatives = 5
test_size = 0.2
random_seed = 42

# Load and clean data
df = pd.read_csv(input_path)
df["text"] = df["text"].astype(str).str.strip()
df["relation_type"] = df["relation_type"].astype(str).str.strip()

# Remove rows with invalid EDUs or relation types
df = df[
    (df["text"] != "") & (df["text"] != "0") &
    (df["relation_type"] != "") & (df["relation_type"] != "0")
]

# Group by dialogue
grouped = df.groupby(["doc", "dialogue_num"])
all_dialogues = []
relation_counter = Counter()

for (doc_id, dlg_num), group in tqdm(grouped):
    group_valid = group[group["relation_type"].isin(allowed_relations)]
    if group_valid.empty:
        continue

    sorted_group = group.sort_values(by="span_end")
    edus = sorted_group["text"].tolist()
    if len(edus) < 3:
        continue

    dominant_relation = group_valid["relation_type"].value_counts().idxmax()
    relation_counter.update([dominant_relation])

    all_dialogues.append({
        "doc_id": f"{doc_id}_{dlg_num}",
        "edus": edus,
        "relation_type": dominant_relation
    })

# Split into train/test dialogues
train_dialogues, test_dialogues = train_test_split(
    all_dialogues, test_size=test_size, random_state=random_seed
)

# Expand to positive + 5 negative samples
def expand_examples(dialogues):
    data = []
    for dlg in tqdm(dialogues, desc="Expanding examples"):
        gold = " </s> ".join(dlg["edus"])
        data.append({
            "doc_id": dlg["doc_id"] + "_gold",
            "input": gold,
            "label": 1,
            "relation_type": dlg["relation_type"]
        })

        seen = {gold}
        for i in range(num_negatives):
            neg = dlg["edus"].copy()
            attempts = 0
            while True:
                random.shuffle(neg)
                neg_str = " </s> ".join(neg)
                if neg_str not in seen:
                    seen.add(neg_str)
                    break
                attempts += 1
                if attempts > 10:
                    break  # avoid infinite loops

            data.append({
                "doc_id": f"{dlg['doc_id']}_neg{i+1}",
                "input": neg_str,
                "label": 0,
                "relation_type": dlg["relation_type"]
            })
    return data

train_data = expand_examples(train_dialogues)
test_data = expand_examples(test_dialogues)

# Save JSONL files
def save_jsonl(data, path):
    with open(path, "w", encoding="utf-8") as f:
        for ex in data:
            f.write(json.dumps(ex) + "\n")

train_path = f"{output_prefix}_train.jsonl"
test_path = f"{output_prefix}_test.jsonl"
rel_dist_path = f"{output_prefix}_relation_distribution.csv"

save_jsonl(train_data, train_path)
save_jsonl(test_data, test_path)

# Save relation distribution
rel_df = pd.DataFrame(relation_counter.items(), columns=["relation_type", "count"])
rel_df.to_csv(rel_dist_path, index=False)

# Conditional download (avoid long stalls)
def get_size_mb(path):
    return round(os.path.getsize(path) / (1024 * 1024), 2)

print(f"\n Train file size: {get_size_mb(train_path)} MB")
print(f"Test file size:  {get_size_mb(test_path)} MB")

if get_size_mb(train_path) < 100:
    files.download(train_path)
if get_size_mb(test_path) < 100:
    files.download(test_path)
if get_size_mb(rel_dist_path) < 10:
    files.download(rel_dist_path)

# === Summary ===
print(f"Train examples: {len(train_data)}")
print(f"Test examples:  {len(test_data)}")

Expanding examples: 100%|██████████| 188/188 [00:00<00:00, 36113.08it/s]


 Train file size: 1.48 MB
Test file size:  0.37 MB


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Train examples: 4500
Test examples:  1128


# RoBERTa Sentence Ordering Fine-tuning

In [ ]:
import json
import random
import torch
import numpy as np
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import (
    RobertaTokenizer,
    RobertaForSequenceClassification,
    get_scheduler
)
from torch.optim import AdamW

# Load and Prepare Dataset
class RobertaRerankDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=512):
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.inputs = [item["input"] for item in data]
        self.labels = [item["label"] for item in data]

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        tokenized = self.tokenizer(
            self.inputs[idx],
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt"
        )
        return {
            "input_ids": tokenized["input_ids"].squeeze(0),
            "attention_mask": tokenized["attention_mask"].squeeze(0),
            "labels": torch.tensor(self.labels[idx], dtype=torch.long)
        }

# Load training data
with open("/content/roberta_rerank_train.jsonl", "r", encoding="utf-8") as f:
    train_data = [json.loads(line) for line in f]

# Tokenizer and Model
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=2)

# Dataset and Dataloader
train_dataset = RobertaRerankDataset(train_data, tokenizer)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

# Optimizer and Scheduler
optimizer = AdamW(model.parameters(), lr=2e-5)
num_epochs = 3
num_training_steps = num_epochs * len(train_loader)
lr_scheduler = get_scheduler(
    "linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training Loop
print(f"Total training pairs: {len(train_dataset)}")
model.train()
for epoch in range(num_epochs):
    total_loss = 0
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}")
    for batch in pbar:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        total_loss += loss.item()
        pbar.set_postfix({"loss": f"{loss.item():.4f}"})
    print(f"Epoch {epoch+1} Avg Loss: {total_loss / len(train_loader):.4f}")

# Save model and tokenizer
output_dir = "/content/roberta_reranker_best"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
print(f"Model and tokenizer saved to {output_dir}")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

...

Epoch 1 Avg Loss: 0.4589


Epoch 2: 100%|██████████| 563/563 [06:40<00:00,  1.41it/s, loss=0.1462]


Epoch 2 Avg Loss: 0.4429


Epoch 3: 100%|██████████| 563/563 [06:40<00:00,  1.41it/s, loss=0.0930]


Epoch 3 Avg Loss: 0.3684
Model and tokenizer saved to /content/roberta_reranker_best


# Fine-tuned RoBERTa Sentence Ordering Evaluation

In [ ]:
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from scipy.stats import spearmanr
from collections import defaultdict
from tqdm import tqdm
import json

# Load and group test data
test_file_path = "roberta_rerank_test.jsonl"
with open(test_file_path, "r", encoding="utf-8") as f:
    lines = [json.loads(line) for line in f]

# Group by doc_id prefix
grouped = defaultdict(list)
for item in lines:
    base_id = item["doc_id"].rsplit("_", 1)[0]
    grouped[base_id].append(item)

# Load model and tokenizer
model_path = "/content/roberta_reranker_best"  # finetuned checkpoint
tokenizer = RobertaTokenizer.from_pretrained(model_path)
model = RobertaForSequenceClassification.from_pretrained(model_path)
model = model.to("cuda" if torch.cuda.is_available() else "cpu")
model.eval()

# Evaluate
all_scores = []
relation_scores = defaultdict(list)
examples = []

# NEW: collect per-dialogue arrays for export later
roberta_dialogue_ids_fine = []
roberta_relations_fine    = []
roberta_rhos_finetuned    = []
skipped = 0

print(f"Evaluating {len(grouped)} dialogues...\n")

for doc_id, candidates in tqdm(grouped.items(), desc="Reranking"):
    relation = candidates[0]["relation_type"]

    inputs = [c["input"] for c in candidates]
    labels = [c["label"] for c in candidates]  # one gold = 1, others = 0

    encodings = tokenizer(
        inputs,
        padding="longest",
        truncation=True,
        max_length=512,
        return_tensors="pt"
    ).to(model.device)

    with torch.no_grad():
        outputs = model(**encodings)
        probs = torch.softmax(outputs.logits, dim=1)[:, 1]  # score for label=1

    top_idx = torch.argmax(probs).item()
    top_input = inputs[top_idx]
    gold_input = inputs[labels.index(1)]

    pred_edus = top_input.split(" </s> ")
    gold_edus = gold_input.split(" </s> ")

    # Only evaluate when there are at least 2 overlapping EDUs
    overlap = [e for e in pred_edus if e in gold_edus]
    if len(overlap) < 2:
        skipped += 1
        continue

    # Spearman between positions in predicted vs. gold order (restricted to overlap)
    try:
        pred_positions = list(range(len(overlap)))                 # 0..k-1 (pred order)
        gold_positions = [gold_edus.index(e) for e in overlap]     # indices in gold
        rho, _ = spearmanr(pred_positions, gold_positions)
        if rho != rho:  # NaN check
            skipped += 1
            continue
    except Exception:
        skipped += 1
        continue

    all_scores.append(rho)
    relation_scores[relation].append(rho)

    roberta_dialogue_ids_fine.append(doc_id)
    roberta_relations_fine.append(relation)
    roberta_rhos_finetuned.append(float(rho))

    examples.append({
        "doc_id": doc_id,
        "relation": relation,
        "match": pred_edus == gold_edus,
        "pred": pred_edus,
        "gold": gold_edus,
        "rho": float(rho),
    })

# Print a few examples (limit to keep output tidy)
print("\nPrinting Reranking Examples (first 5):\n")
for ex in examples[:5]:
    print(f"Dialogue ID: {ex['doc_id']}")
    print(f"Relation Type: {ex['relation']}")
    print(f"Gold Ranked First? {'Yes' if ex['match'] else 'No'}  |  ρ={ex['rho']:.4f}\n")
    print("Top Prediction:")
    for edu in ex["pred"]:
        print(f"   {edu}")
    print("\nGold Sequence:")
    for edu in ex["gold"]:
        print(f"   {edu}")
    print("\n" + "-" * 60 + "\n")

# Print scores
avg_score = sum(all_scores) / len(all_scores) if all_scores else 0.0
print(f"\nOverall Average Spearman’s ρ: {avg_score:.4f}")
print(f"Evaluated: {len(all_scores)}, Skipped (overlap < 2 or NaN): {skipped}\n")

print("--- Spearman’s ρ by Relation Type ---")
for rel, scores in relation_scores.items():
    rel_avg = sum(scores) / len(scores)
    print(f"{rel:<25} n={len(scores):<4} | Mean ρ: {rel_avg:.4f}")

Evaluating 188 dialogues...



Reranking: 100%|██████████| 188/188 [00:06<00:00, 30.55it/s]


Printing Reranking Examples (first 5):

Dialogue ID: pilot03_82.0
Relation Type: Comment
Gold Ranked First? No  |  ρ=-0.6000

Top Prediction:
   Sure, Joel.
   i'm hungry :D
   thx
   And I get more mutton.
   joel gets more wheat for the weetos and wheat crunchie producers :D

Gold Sequence:
   joel gets more wheat for the weetos and wheat crunchie producers :D
   And I get more mutton.
   Sure, Joel.
   i'm hungry :D
   thx

------------------------------------------------------------

Dialogue ID: s1_league2_game1_404.0
Relation Type: Question_answer_pair
Gold Ranked First? No  |  ρ=-0.8000

Top Prediction:
   No wood, sorry
   Anyone got wood to trade?
   nope
   Yup

Gold Sequence:
   Yup
   Anyone got wood to trade?
   nope
   No wood, sorry

------------------------------------------------------------

Dialogue ID: s2_leagueM_game3_1041.0
Relation Type: Comment
Gold Ranked First? Yes  |  ρ=0.9995

Top Prediction:
   what do you have
   sheep and ore
   all?
   yep
   addtime
  

## Export RoBERTa finetuned per-dialogue CSV

In [ ]:
# Export RoBERTa ORDERING (FINETUNED) per-dialogue CSV
import os, pandas as pd

OUT_PATH = "/content/exports/roberta_ordering_finetuned.csv"
os.makedirs("/content/exports", exist_ok=True)

# Must exist from the eval cell:
assert 'roberta_rhos_finetuned' in globals() and len(roberta_rhos_finetuned) > 0, \
    "Missing 'roberta_rhos_finetuned' — run the RoBERTa finetuned evaluation cell first."

N = len(roberta_rhos_finetuned)

dialogue_ids = globals().get('roberta_dialogue_ids_fine', None)
relations    = globals().get('roberta_relations_fine', None)

if dialogue_ids is None or len(dialogue_ids) != N:
    dialogue_ids = [f"dlg_{i:04d}" for i in range(N)]
if relations is None or len(relations) != N:
    relations = ["Unknown"] * N

# Trim defensively to the shortest length
N = min(N, len(dialogue_ids), len(relations))

df_fin = pd.DataFrame({
    "dialogue_id":   [str(x) for x in dialogue_ids[:N]],
    "relation_type": [str(x) for x in relations[:N]],
    "metric_value":  [float(x) for x in roberta_rhos_finetuned[:N]],
})

df_fin.to_csv(OUT_PATH, index=False)
print(f"Saved → {OUT_PATH}  rows: {len(df_fin)}")
display(df_fin.head(5))

Saved → /content/exports/roberta_ordering_finetuned.csv  rows: 188


,dialogue_id,relation_type,metric_value
0,pilot03_82.0,Comment,-0.600000
1,s1_league2_game1_404.0,Question_answer_pair,-0.800000
2,s2_leagueM_game3_1041.0,Comment,0.999484
3,s1_league3_game6_596.0,Elaboration,0.000000
4,s1_league1_game1_243.0,Comment,1.000000


# RoBERTa Senetence Ordering Baseline Evaluation

In [ ]:
import json, math
from collections import defaultdict
from scipy.stats import spearmanr
from tqdm import tqdm
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification

# Load model and tokeniser (BASELINE)
model_name_or_path = "roberta-base"  # <-- plain baseline (not fine-tuned)
tokenizer = RobertaTokenizer.from_pretrained(model_name_or_path)
model = RobertaForSequenceClassification.from_pretrained(model_name_or_path)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device).eval()

# Load test data
test_path = "/content/roberta_rerank_test.jsonl"
with open(test_path, "r", encoding="utf-8") as f:
    raw_data = [json.loads(line) for line in f]

# Group examples by dialogue ID (doc_id prefix)
dialogues = defaultdict(list)
for entry in raw_data:
    base_id = entry["doc_id"].rsplit("_", 1)[0]
    dialogues[base_id].append(entry)

# Evaluation
all_rhos = []
rel_rhos = defaultdict(list)
examples = []

roberta_rhos_baseline = []
roberta_dialogue_ids_base = []
roberta_relations_base = []

skipped = 0
print(f"Evaluating {len(dialogues)} dialogues...\n")

for dlg_id, entries in tqdm(dialogues.items(), desc="Reranking (baseline)"):
    relation = entries[0]["relation_type"]

    # Collect candidate inputs and find gold index
    input_texts = [it["input"] for it in entries]
    gold_index = next((i for i, it in enumerate(entries) if it.get("label", 0) == 1), None)
    if gold_index is None:
        skipped += 1
        continue

    # Tokenize and score candidates with baseline model
    enc = tokenizer(
        input_texts,
        padding="longest",            # match finetuned settings
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )
    enc = {k: v.to(device) for k, v in enc.items()}

    with torch.no_grad():
        logits = model(**enc).logits
        scores = logits.softmax(dim=-1)[:, 1]   # probability for class "1"

    # Pick the top-scoring candidate and compare its EDU order to gold
    top_idx = int(torch.argmax(scores).item())
    pred_edus = input_texts[top_idx].split(" </s> ")
    gold_edus = input_texts[gold_index].split(" </s> ")

    # Restrict to overlapping EDUs (avoid NaN for disjoint or constant ranks)
    overlap = [e for e in pred_edus if e in gold_edus]
    if len(overlap) < 2:
        skipped += 1
        continue

    try:
        # Positions in predicted order vs positions in gold order (for the same overlapping EDUs)
        pred_positions = list(range(len(overlap)))
        gold_positions = [gold_edus.index(e) for e in overlap]
        rho, _ = spearmanr(pred_positions, gold_positions)
        if rho != rho:  # NaN
            skipped += 1
            continue
    except Exception:
        skipped += 1
        continue

    # Aggregate scores
    all_rhos.append(rho)
    rel_rhos[relation].append(rho)

    # Record per-dialogue outputs for export
    roberta_rhos_baseline.append(float(rho))
    roberta_dialogue_ids_base.append(dlg_id)
    roberta_relations_base.append(relation)

    # Keep a few examples for inspection
    if len(examples) < 5:
        examples.append({
            "dlg_id": dlg_id,
            "relation": relation,
            "correct": (top_idx == gold_index),
            "rho": float(rho),
            "top_pred": pred_edus,
            "gold_seq": gold_edus
        })

# Print a few examples
print("\nSample Reranking Examples (first 5):\n")
for ex in examples:
    print(f"Dialogue ID: {ex['dlg_id']}")
    print(f"Relation Type: {ex['relation']}")
    print(f"Gold Ranked First? {'Yes' if ex['correct'] else 'No'}  |  ρ={ex['rho']:.4f}\n")
    print("Top Prediction:")
    for line in ex['top_pred']:
        print(f"  {line}")
    print("\nGold Sequence:")
    for line in ex['gold_seq']:
        print(f"  {line}")
    print("\n" + "-" * 60 + "\n")

# Results
avg_rho = (sum(all_rhos) / len(all_rhos)) if all_rhos else 0.0
print(f"\nOverall Average Spearman’s ρ: {avg_rho:.4f}")
print(f"Evaluated: {len(all_rhos)}, Skipped: {skipped}\n")

print("Spearman’s ρ by Relation Type")
for rel, scores in rel_rhos.items():
    print(f"{rel:<20} n={len(scores):<4} | Mean ρ: {sum(scores)/len(scores):.4f}")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Evaluating 188 dialogues...



Reranking (baseline): 100%|██████████| 188/188 [00:05<00:00, 31.88it/s]


Sample Reranking Examples (first 5):

Dialogue ID: pilot03_82.0
Relation Type: Comment
Gold Ranked First? No  |  ρ=-0.8000

Top Prediction:
  i'm hungry :D
  thx
  Sure, Joel.
  joel gets more wheat for the weetos and wheat crunchie producers :D
  And I get more mutton.

Gold Sequence:
  joel gets more wheat for the weetos and wheat crunchie producers :D
  And I get more mutton.
  Sure, Joel.
  i'm hungry :D
  thx

------------------------------------------------------------

Dialogue ID: s1_league2_game1_404.0
Relation Type: Question_answer_pair
Gold Ranked First? No  |  ρ=0.4000

Top Prediction:
  Yup
  nope
  No wood, sorry
  Anyone got wood to trade?

Gold Sequence:
  Yup
  Anyone got wood to trade?
  nope
  No wood, sorry

------------------------------------------------------------

Dialogue ID: s2_leagueM_game3_1041.0
Relation Type: Comment
Gold Ranked First? Yes  |  ρ=0.9995

Top Prediction:
  what do you have
  sheep and ore
  all?
  yep
  addtime
  how much ore
  2
  ok 2 or

## RoBERTa sentence ordering (significance)

In [ ]:
# Export RoBERTa ORDERING (BASELINE) per-dialogue CSV
import os, pandas as pd

BASELINE_OUT = "/content/exports/roberta_ordering_baseline.csv"
FINETUNED_IN = "/content/exports/roberta_ordering_finetuned.csv"  # used if present for consistent IDs
os.makedirs("/content/exports", exist_ok=True)

# Must exist from your baseline eval cell:
assert 'roberta_rhos_baseline' in globals() and len(roberta_rhos_baseline) > 0, \
    "Missing 'roberta_rhos_baseline' — run the RoBERTa BASELINE evaluation first."

def _len(x):
    try: return len(x)
    except: return 0

if os.path.exists(FINETUNED_IN):
    fin = pd.read_csv(FINETUNED_IN)
    N = min(len(fin), len(roberta_rhos_baseline))
    df_base = pd.DataFrame({
        "dialogue_id":   fin["dialogue_id"].iloc[:N].astype(str).tolist(),
        "relation_type": fin["relation_type"].iloc[:N].astype(str).tolist(),
        "metric_value":  [float(x) for x in roberta_rhos_baseline[:N]],
    })
else:
    ids  = globals().get("roberta_dialogue_ids_base", None)
    rels = globals().get("roberta_relations_base", None)
    N = len(roberta_rhos_baseline)
    if ids is None or _len(ids) != N:
        ids = [f"dlg_{i:04d}" for i in range(N)]
    if rels is None or _len(rels) != N:
        rels = ["Unknown"] * N
    df_base = pd.DataFrame({
        "dialogue_id":   [str(x) for x in ids[:N]],
        "relation_type": [str(x) for x in rels[:N]],
        "metric_value":  [float(x) for x in roberta_rhos_baseline[:N]],
    })

df_base.to_csv(BASELINE_OUT, index=False)
print(f"Saved → {BASELINE_OUT}  rows: {len(df_base)}")
display(df_base.head(5))

Saved → /content/exports/roberta_ordering_baseline.csv  rows: 188


,dialogue_id,relation_type,metric_value
0,pilot03_82.0,Comment,-0.800000
1,s1_league2_game1_404.0,Question_answer_pair,0.400000
2,s2_leagueM_game3_1041.0,Comment,0.999484
3,s1_league3_game6_596.0,Elaboration,1.000000
4,s1_league1_game1_243.0,Comment,1.000000


## RoBERTa ORDERING significance

In [ ]:
# RoBERTa ORDERING significance (paired Wilcoxon + CI + by-relation)
import os, math, numpy as np, pandas as pd
from scipy.stats import wilcoxon

BASELINE   = "/content/exports/roberta_ordering_baseline.csv"
FINETUNED  = "/content/exports/roberta_ordering_finetuned.csv"
OUT_PREFIX = "/content/exports/roberta_ordering"
MIN_N      = 8
METRIC     = "ρ"

def _bootstrap_mean_diff_CI(x, y, n_boot=10000, seed=42, alpha=0.05):
    rng = np.random.default_rng(seed)
    diffs = np.asarray(x) - np.asarray(y)
    idx = rng.integers(0, len(diffs), size=(n_boot, len(diffs)))
    boot_means = diffs[idx].mean(axis=1)
    return float(np.quantile(boot_means, alpha/2)), float(np.quantile(boot_means, 1 - alpha/2))

# Load & merge
assert os.path.exists(BASELINE), f"Missing: {BASELINE}"
assert os.path.exists(FINETUNED), f"Missing: {FINETUNED}"
b = pd.read_csv(BASELINE)
f = pd.read_csv(FINETUNED)
for req in ["dialogue_id","relation_type","metric_value"]:
    assert req in b.columns and req in f.columns, f"Missing '{req}' in one of the CSVs."

df = pd.merge(b, f, on="dialogue_id", suffixes=("_baseline","_finetuned"))
df["relation_type"] = df.get("relation_type_finetuned", df.get("relation_type_baseline","Unknown"))
df = df[["dialogue_id","relation_type","metric_value_baseline","metric_value_finetuned"]].dropna()
assert len(df) > 0, "No overlapping dialogues after merge."

if np.allclose(df["metric_value_baseline"], df["metric_value_finetuned"]):
    raise ValueError("Baseline and finetuned arrays are identical — check that you evaluated true baseline vs finetuned.")

# Overall tests
x = df["metric_value_finetuned"].to_numpy()
y = df["metric_value_baseline"].to_numpy()
w_stat, w_p = wilcoxon(x, y, zero_method="wilcox", alternative="two-sided")
w_stat_ch, w_p_ch = wilcoxon(x, np.zeros_like(x), zero_method="wilcox", alternative="greater")
ci_lo, ci_hi = _bootstrap_mean_diff_CI(x, y)

overall = pd.DataFrame([{
    "n_dialogues": len(df),
    f"baseline_mean_{METRIC}": float(y.mean()),
    f"finetuned_mean_{METRIC}": float(x.mean()),
    "mean_diff": float(x.mean() - y.mean()),
    "boot_CI_low": ci_lo, "boot_CI_high": ci_hi,
    "wilcoxon_stat": float(w_stat), "wilcoxon_p": float(w_p),
    f"chance_test_p(finetuned>{0})": float(w_p_ch),
}])

# By-relation tests
rows = []
for rel, grp in df.groupby("relation_type"):
    n = len(grp)
    base = grp["metric_value_baseline"].to_numpy()
    fine = grp["metric_value_finetuned"].to_numpy()
    mean_b, mean_f = float(base.mean()), float(fine.mean())
    diff = float(mean_f - mean_b)
    if n >= MIN_N:
        ws, wp = wilcoxon(fine, base, zero_method="wilcox", alternative="two-sided")
        ws_c, wp_c = wilcoxon(fine, np.zeros_like(fine), zero_method="wilcox", alternative="greater")
        p_pair, p_chance = float(wp), float(wp_c)
    else:
        p_pair = p_chance = ""
    rows.append({
        "relation_type": rel, "n": int(n),
        f"baseline_mean_{METRIC}": mean_b,
        f"finetuned_mean_{METRIC}": mean_f,
        "mean_diff": diff,
        "wilcoxon_p(baseline_vs_finetuned)": p_pair,
        f"chance_test_p(finetuned>{0})": p_chance
    })
byrel = pd.DataFrame(rows).sort_values(["n","relation_type"], ascending=[False, True])

# Save + display
os.makedirs(os.path.dirname(OUT_PREFIX), exist_ok=True)
overall.to_csv(f"{OUT_PREFIX}_significance_overall.csv", index=False)
byrel.to_csv(f"{OUT_PREFIX}_significance_by_relation.csv", index=False)
display(overall); display(byrel)

r = overall.iloc[0]
print(
  f"Significance (RoBERTa sentence ordering). The improvement from baseline ({METRIC} = {r[f'baseline_mean_{METRIC}']:.4f}) "
  f"to fine-tuned ({METRIC} = {r[f'finetuned_mean_{METRIC}']:.4f}) was statistically significant "
  f"(Wilcoxon signed-rank, p = {r['wilcoxon_p']:.3g}; 95% CI for mean difference: "
  f"[{r['boot_CI_low']:.3f}, {r['boot_CI_high']:.3f}]). Fine-tuned performance was also significantly above chance "
  f"({METRIC} > 0; p = {r[f'chance_test_p(finetuned>{0})']:.3g})."
)

,n_dialogues,baseline_mean_ρ,finetuned_mean_ρ,mean_diff,boot_CI_low,boot_CI_high,wilcoxon_stat,wilcoxon_p,chance_test_p(finetuned>0)
0,188,0.148702,0.466453,0.317751,0.200763,0.434309,3191.5,5.511589e-07,1.935549e-18


,relation_type,n,baseline_mean_ρ,finetuned_mean_ρ,mean_diff,wilcoxon_p(baseline_vs_finetuned),chance_test_p(finetuned>0)
4,Question_answer_pair,110,0.105017,0.487006,0.381989,0.000012,0.0
0,Comment,47,0.176711,0.430617,0.253906,0.024557,0.000026
2,Elaboration,15,0.187909,0.341896,0.153987,0.422678,0.025115
1,Continuation,11,0.332922,0.552310,0.219387,0.425781,0.010254
3,Explanation,5,0.323567,0.535908,0.212342,,


Significance (RoBERTa sentence ordering). The improvement from baseline (ρ = 0.1487) to fine-tuned (ρ = 0.4665) was statistically significant (Wilcoxon signed-rank, p = 5.51e-07; 95% CI for mean difference: [0.201, 0.434]). Fine-tuned performance was also significantly above chance (ρ > 0; p = 1.94e-18).


# BART NSP Preprocessing

In [ ]:
import pandas as pd
import json
import random
from collections import defaultdict
from google.colab import files

# Load and clean the dataset
df = pd.read_csv("incoming_base_spect.csv")

# Drop rows with missing or invalid values
df = df[df["text"].notna() & df["relation_type"].notna() & df["incoming_relation_id"].notna()]
df = df[
    (df["text"].astype(str).str.strip() != "0") &
    (df["relation_type"].astype(str).str.strip() != "0")
]

# Group EDUs by dialogue
dialogues = defaultdict(list)
for _, row in df.iterrows():
    dlg_id = row["dialogue_num"] if "dialogue_num" in df.columns else row["doc"]
    dialogues[dlg_id].append({
        "text": row["text"].strip(),
        "relation": row["relation_type"].strip()
    })

# Shuffle and split dialogues
dialogue_ids = list(dialogues.keys())
random.seed(42)
random.shuffle(dialogue_ids)
n = len(dialogue_ids)

split_map = {
    "train": dialogue_ids[:int(0.8 * n)],
    "dev": dialogue_ids[int(0.8 * n):int(0.9 * n)],
    "test": dialogue_ids[int(0.9 * n):]
}

# Create NSP-style pairs
for split, dlg_ids in split_map.items():
    examples = []
    for dlg_id in dlg_ids:
        edus = dialogues[dlg_id]
        if len(edus) < 3:
            continue  # skip short dialogues

        for i in range(len(edus) - 1):
            context = " <EDU_SEP> ".join(e["text"] for e in edus[:i+1])
            target = edus[i + 1]["text"]
            rel = edus[i + 1]["relation"]

            # Positive example
            examples.append({
                "input": context + " <SEP> " + target,
                "label": 1,
                "relation": rel
            })

            # Negative example: random target from another dialogue
            neg_dlg_id = random.choice([d for d in dialogue_ids if d != dlg_id])
            neg_edus = dialogues[neg_dlg_id]
            neg_target = random.choice(neg_edus)["text"]
            examples.append({
                "input": context + " <SEP> " + neg_target,
                "label": 0,
                "relation": rel
            })

    # Save to file
    out_file = f"bart_nsp_{split}.jsonl"
    with open(out_file, "w", encoding="utf-8") as f:
        for ex in examples:
            f.write(json.dumps(ex) + "\n")
    print(f"Saved {len(examples)} examples to {out_file}")
    files.download(out_file)

Saved 15896 examples to bart_nsp_train.jsonl


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Saved 1890 examples to bart_nsp_dev.jsonl


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Saved 1776 examples to bart_nsp_test.jsonl


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# BART NSP Fine-tuning

In [ ]:
import json
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BartTokenizer, BartForSequenceClassification
from torch.optim import AdamW
from tqdm import tqdm
import os

# Parameters
train_file = "bart_nsp_train.jsonl"
dev_file = "bart_nsp_dev.jsonl"
save_dir = "bart_nsp_model"
num_epochs = 3
batch_size = 4
lr = 2e-5
max_length = 512
gradient_accumulation_steps = 4

# Custom Tokens
special_tokens = {"additional_special_tokens": ["[CTX_SEP]", "[EDU_SEP]"]}

# Dataset
class BARTNSPDataset(Dataset):
    def __init__(self, path, tokenizer):
        self.data = []
        with open(path, "r", encoding="utf-8") as f:
            for line in f:
                item = json.loads(line)
                self.data.append(item)
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        encodings = self.tokenizer(
            item["input"],
            truncation=True,
            padding="max_length",
            max_length=max_length,
            return_tensors="pt"
        )
        return {
            "input_ids": encodings["input_ids"].squeeze(),
            "attention_mask": encodings["attention_mask"].squeeze(),
            "labels": torch.tensor(item["label"], dtype=torch.long)
        }

# Tokeniser and Model
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")
tokenizer.add_special_tokens(special_tokens)

model = BartForSequenceClassification.from_pretrained("facebook/bart-base", num_labels=2)
model.resize_token_embeddings(len(tokenizer))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# DataLoaders
train_dataset = BARTNSPDataset(train_file, tokenizer)
dev_dataset = BARTNSPDataset(dev_file, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=batch_size)

optimizer = AdamW(model.parameters(), lr=lr)
best_dev_loss = float("inf")

# Training Loop
for epoch in range(1, num_epochs + 1):
    model.train()
    train_loss = 0.0

    for step, batch in enumerate(tqdm(train_loader, desc=f"Epoch {epoch}")):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss / gradient_accumulation_steps
        loss.backward()

        if (step + 1) % gradient_accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        train_loss += loss.item()

    avg_train_loss = train_loss / len(train_loader)
    print(f"Epoch {epoch}: Training Loss = {avg_train_loss:.4f}")

    # Evaluation
    model.eval()
    dev_loss = 0.0
    with torch.no_grad():
        for batch in dev_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            dev_loss += outputs.loss.item()

    avg_dev_loss = dev_loss / len(dev_loader)
    print(f"Epoch {epoch}: Dev Loss = {avg_dev_loss:.4f}")

    if avg_dev_loss < best_dev_loss:
        best_dev_loss = avg_dev_loss
        os.makedirs(save_dir, exist_ok=True)
        model.save_pretrained(save_dir)
        tokenizer.save_pretrained(save_dir)
        print(f"Best model saved (Epoch {epoch})")

print("Training complete.")

Some weights of BartForSequenceClassification were not initialized from the model checkpoint at facebook/bart-base and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 1: 100%|██████████| 3974/3974 [29:05<00:00,  2.28it/s]


Epoch 1: Training Loss = 0.1686
Epoch 1: Dev Loss = 0.6493
Best model saved (Epoch 1)


Epoch 2: 100%|██████████| 3974/3974 [29:01<00:00,  2.28it/s]


Epoch 2: Training Loss = 0.1572
Epoch 2: Dev Loss = 0.6075
Best model saved (Epoch 2)


Epoch 3: 100%|██████████| 3974/3974 [29:00<00:00,  2.28it/s]


Epoch 3: Training Loss = 0.1454
Epoch 3: Dev Loss = 0.6019
Best model saved (Epoch 3)
Training complete.


# Fine-tuned BART NSP Evaluation

In [ ]:
import json
import torch
from transformers import AutoConfig, BartTokenizer, BartForSequenceClassification
from sklearn.metrics import accuracy_score, f1_score
from collections import defaultdict
from torch.utils.data import Dataset, DataLoader

# Parameters
dev_file   = "bart_nsp_dev.jsonl"
model_path = "bart_nsp_model"
batch_size = 4
max_length = 512

# Load Fine-tuned Model and Tokeniser (force binary labels)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = BartTokenizer.from_pretrained(model_path)

cfg = AutoConfig.from_pretrained(model_path)
cfg.num_labels = 2
cfg.id2label = {0: "NEG", 1: "POS"}
cfg.label2id = {"NEG": 0, "POS": 1}

model = BartForSequenceClassification.from_pretrained(
    model_path,
    config=cfg,
    ignore_mismatched_sizes=True  # avoids head-shape errors if checkpoint had wrong num_labels
).to(device)
model.eval()

# Dataset Definition (no None returns)
class BARTNSPDataset(Dataset):
    def __init__(self, path, tokenizer):
        self.data = []
        with open(path, "r", encoding="utf-8") as f:
            for line in f:
                self.data.append(json.loads(line))
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item   = self.data[idx]
        text   = item.get("input", "")
        label  = int(item.get("label", 0))  # ensure int 0/1
        rel    = item.get("relation") or item.get("relation_type") or "Unknown"
        dlg_id = item.get("dialogue_id") or item.get("doc_id") or f"dlg_auto_{idx:06d}"

        enc = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=max_length,
            return_tensors="pt"
        )

        return {
            "input_ids": enc["input_ids"].squeeze(0),
            "attention_mask": enc["attention_mask"].squeeze(0),
            "labels": torch.tensor(label, dtype=torch.long),
            "relation": str(rel),
            "dialogue_id": str(dlg_id),
            "input_text": text,
        }

# Load Dataset
dev_dataset = BARTNSPDataset(dev_file, tokenizer)
dev_loader = DataLoader(dev_dataset, batch_size=batch_size, shuffle=False)

# Inference
all_preds, all_labels, all_rels = [], [], []
all_inputs, all_ids = [], []
per_dialogue = defaultdict(lambda: {"y_true": [], "y_pred": [], "relation": None})

with torch.no_grad():
    for batch in dev_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        logits = model(input_ids=input_ids, attention_mask=attention_mask).logits
        preds = torch.argmax(logits, dim=1)

        labels_list = labels.cpu().tolist()
        preds_list  = preds.cpu().tolist()
        rels_list   = list(batch["relation"])
        ids_list    = list(batch["dialogue_id"])
        inputs_list = list(batch["input_text"])

        for y, yhat, rel, dlg, txt in zip(labels_list, preds_list, rels_list, ids_list, inputs_list):
            all_labels.append(y)
            all_preds.append(yhat)
            all_rels.append(rel)
            all_inputs.append(txt)
            all_ids.append(dlg)

            per_dialogue[dlg]["y_true"].append(y)
            per_dialogue[dlg]["y_pred"].append(yhat)
            if per_dialogue[dlg]["relation"] is None:
                per_dialogue[dlg]["relation"] = rel

# Print all examples by relation (same as your original; keep or trim)
rel_wise = defaultdict(lambda: {"labels": [], "preds": [], "examples": []})
for inp, label, pred, rel in zip(all_inputs, all_labels, all_preds, all_rels):
    rel_wise[rel]["labels"].append(label)
    rel_wise[rel]["preds"].append(pred)
    rel_wise[rel]["examples"].append((inp, label, pred))

print("\n--- All Examples by Relation Type ---")
for rel, data in rel_wise.items():
    print(f"\nRelation: {rel}")
    for inp, label, pred in data["examples"]:
        print(f"Input: {inp}")
        print(f"Gold: {label} | Pred: {pred}\n")

# Evaluation by Relation (pair-level)
print("\n--- Evaluation by Discourse Relation Type ---")
for rel, data in rel_wise.items():
    count = len(data["labels"])
    acc = accuracy_score(data["labels"], data["preds"])
    f1  = f1_score(data["labels"], data["preds"], average="macro")
    print(f"{rel:<25} n={count:<4} | Accuracy: {acc:.3f} | Macro F1: {f1:.3f}")

# Overall Scores (pair-level) + empirical chance
overall_acc = accuracy_score(all_labels, all_preds)
overall_f1  = f1_score(all_labels, all_preds, average="macro")
pos_rate    = (sum(all_labels) / len(all_labels)) if all_labels else 0.0  # use for chance if not 0.5
print(f"\nOverall Accuracy: {overall_acc:.4f}")
print(f"Overall Macro F1 Score: {overall_f1:.4f}")
print(f"Empirical positive rate (chance level): {pos_rate:.3f}")

# Per-dialogue accuracies (needed for significance tests)
records = []
for dlg, d in per_dialogue.items():
    acc = accuracy_score(d["y_true"], d["y_pred"])
    rel = d["relation"] if d["relation"] is not None else "Unknown"
    records.append((dlg, rel, float(acc)))
records.sort(key=lambda x: x[0])

# Variables picked up by the export/significance cells
bart_nsp_dialogue_ids_fine = [r[0] for r in records]
bart_nsp_relations_fine    = [r[1] for r in records]
bart_nsp_acc_finetuned     = [r[2] for r in records]

print(f"\nPer-dialogue entries: {len(bart_nsp_acc_finetuned)} "
      f"(mean acc across dialogues = {sum(bart_nsp_acc_finetuned)/len(bart_nsp_acc_finetuned):.4f})")

You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.


Streaming output truncated to the last 5000 lines.
Gold: 0 | Pred: 0

Input: nope <EDU_SEP> i'll trade 1 for 1 <EDU_SEP> deal <EDU_SEP> anyone else? <SEP> not me
Gold: 1 | Pred: 1

Input: nope <EDU_SEP> i'll trade 1 for 1 <EDU_SEP> deal <EDU_SEP> anyone else? <SEP> can do that
Gold: 0 | Pred: 0

Input: nope <EDU_SEP> i'll trade 1 for 1 <EDU_SEP> deal <EDU_SEP> anyone else? <EDU_SEP> not me <EDU_SEP> i'd hunger for another <EDU_SEP> sorry, <SEP> need my other one
Gold: 1 | Pred: 1

Input: nope <EDU_SEP> i'll trade 1 for 1 <EDU_SEP> deal <EDU_SEP> anyone else? <EDU_SEP> not me <EDU_SEP> i'd hunger for another <EDU_SEP> sorry, <SEP> haha :D
Gold: 0 | Pred: 0

Input: hey everyone <EDU_SEP> Sorry I'm late <EDU_SEP> hello! <EDU_SEP> hello! <EDU_SEP> hello! <EDU_SEP> np. <EDU_SEP> did the wheat colour load properly for everyone? <EDU_SEP> did the wheat colour load properly for everyone? <EDU_SEP> did the wheat colour load properly for everyone? <EDU_SEP> did the wheat colour load properly for

In [ ]:
# Export BART NSP FINETUNED per-dialogue CSV
import os, pandas as pd

OUT_PATH = "/content/exports/bart_nsp_finetuned.csv"
os.makedirs("/content/exports", exist_ok=True)

# Must exist from the eval cell:
assert 'bart_nsp_acc_finetuned' in globals() and len(bart_nsp_acc_finetuned) > 0, \
    "Missing 'bart_nsp_acc_finetuned' — run the fine-tuned NSP eval cell first."

N = len(bart_nsp_acc_finetuned)
dlg_ids = globals().get('bart_nsp_dialogue_ids_fine', None)
rels    = globals().get('bart_nsp_relations_fine', None)

if dlg_ids is None or len(dlg_ids) != N:
    dlg_ids = [f"dlg_{i:04d}" for i in range(N)]
if rels is None or len(rels) != N:
    rels = ["Unknown"] * N

N = min(N, len(dlg_ids), len(rels))
df_fin = pd.DataFrame({
    "dialogue_id":   [str(x) for x in dlg_ids[:N]],
    "relation_type": [str(x) for x in rels[:N]],
    "metric_value":  [float(x) for x in bart_nsp_acc_finetuned[:N]],  # accuracy in [0,1]
})

df_fin.to_csv(OUT_PATH, index=False)
print(f"Saved → {OUT_PATH}  rows: {len(df_fin)}")
display(df_fin.head(5))

Saved → /content/exports/bart_nsp_finetuned.csv  rows: 1890


,dialogue_id,relation_type,metric_value
0,dlg_auto_000000,Question_answer_pair,1.0
1,dlg_auto_000001,Question_answer_pair,1.0
2,dlg_auto_000002,Q_Elab,1.0
3,dlg_auto_000003,Q_Elab,0.0
4,dlg_auto_000004,Question_answer_pair,0.0


# BART NSP Baseline Evaluation

In [ ]:
import json
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BartTokenizer, BartForSequenceClassification
from sklearn.metrics import accuracy_score, f1_score
from collections import defaultdict, Counter

# Parameters
test_file = "bart_nsp_test.jsonl"
batch_size = 4
max_length = 512

# Special tokens
special_tokens = {"additional_special_tokens": ["[CTX_SEP]", "[EDU_SEP]"]}

# Dataset Class (ensure no None fields)
class BARTNSPDataset(Dataset):
    def __init__(self, path, tokenizer):
        self.data = []
        with open(path, "r", encoding="utf-8") as f:
            for line in f:
                self.data.append(json.loads(line))
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        text = item.get("input", "")
        label = int(item.get("label", 0))
        relation = item.get("relation") or item.get("relation_type") or "Unknown"
        # robust dialogue id
        dlg_id = item.get("dialogue_id") or item.get("doc_id") or f"dlg_base_{idx:06d}"

        encodings = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=max_length,
            return_tensors="pt"
        )
        return {
            "input_ids": encodings["input_ids"].squeeze(0),
            "attention_mask": encodings["attention_mask"].squeeze(0),
            "labels": torch.tensor(label, dtype=torch.long),
            "relation": str(relation),
            "dialogue_id": str(dlg_id),
            "input_text": text
        }

# Load Tokeniser and Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")
tokenizer.add_special_tokens(special_tokens)

# Binary NSP head
model = BartForSequenceClassification.from_pretrained("facebook/bart-base", num_labels=2)
model.resize_token_embeddings(len(tokenizer))
model.to(device).eval()

# Load Dataset
test_dataset = BARTNSPDataset(test_file, tokenizer)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Evaluation
all_preds, all_labels, all_rels, all_inputs, all_ids = [], [], [], [], []
per_dialogue = defaultdict(lambda: {"y_true": [], "y_pred": [], "relation": None})

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        logits = model(input_ids=input_ids, attention_mask=attention_mask).logits
        preds = torch.argmax(logits, dim=-1)  # robust decision rule

        labels_list = labels.cpu().tolist()
        preds_list  = preds.cpu().tolist()
        rels_list   = list(batch["relation"])
        ids_list    = list(batch["dialogue_id"])
        inputs_list = list(batch["input_text"])

        # collect global + per-dialogue
        for y, yhat, rel, dlg, txt in zip(labels_list, preds_list, rels_list, ids_list, inputs_list):
            all_labels.append(y)
            all_preds.append(yhat)
            all_rels.append(rel)
            all_inputs.append(txt)
            all_ids.append(dlg)

            per_dialogue[dlg]["y_true"].append(y)
            per_dialogue[dlg]["y_pred"].append(yhat)
            if per_dialogue[dlg]["relation"] is None:
                per_dialogue[dlg]["relation"] = rel

# Class distribution check
print("Predicted Label Distribution:", Counter(all_preds))
print("True Label Distribution:", Counter(all_labels))

# Print examples with relation types (trimmed)
print("\n--- Evaluation Examples by Relation Type ---")
rel_wise = defaultdict(lambda: {"preds": [], "labels": [], "examples": []})
for inp, pred, label, rel in zip(all_inputs, all_preds, all_labels, all_rels):
    rel_wise[rel]["preds"].append(pred)
    rel_wise[rel]["labels"].append(label)
    if len(rel_wise[rel]["examples"]) < 3:
        rel_wise[rel]["examples"].append((inp, label, pred))

for rel, vals in rel_wise.items():
    print(f"\nRelation: {rel}")
    for inp, label, pred in vals["examples"]:
        print(f"Input: {inp}")
        print(f"Gold: {label} | Pred: {pred}\n")

# Scores by relation type (pair-level)
print("\n--- Evaluation by Discourse Relation Type (Macro F1) ---")
for rel, vals in rel_wise.items():
    labels = vals["labels"]
    preds = vals["preds"]
    count = len(labels)
    acc = accuracy_score(labels, preds)
    macro_f1 = f1_score(labels, preds, average="macro")
    print(f"{rel:<25} n={count:<4} | Accuracy: {acc:.3f} | Macro F1: {macro_f1:.3f}")

# Overall scores (pair-level) + empirical chance
overall_acc = accuracy_score(all_labels, all_preds)
overall_macro_f1 = f1_score(all_labels, all_preds, average="macro")
pos_rate = (sum(all_labels) / len(all_labels)) if all_labels else 0.0
print(f"\nOverall Accuracy: {overall_acc:.4f}")
print(f"Overall Macro F1 Score: {overall_macro_f1:.4f}")
print(f"Empirical positive rate (chance level): {pos_rate:.3f}")

# Per-dialogue accuracies (for significance tests) ===
records = []
for dlg, d in per_dialogue.items():
    acc = accuracy_score(d["y_true"], d["y_pred"])
    rel = d["relation"] if d["relation"] is not None else "Unknown"
    records.append((dlg, rel, float(acc)))
records.sort(key=lambda x: x[0])

# Exportable variables (baseline)
bart_nsp_dialogue_ids_base = [r[0] for r in records]
bart_nsp_relations_base    = [r[1] for r in records]
bart_nsp_acc_baseline      = [r[2] for r in records]

print(f"\nPer-dialogue entries (baseline): {len(bart_nsp_acc_baseline)} "
      f"(mean acc across dialogues = {sum(bart_nsp_acc_baseline)/len(bart_nsp_acc_baseline):.4f})")

Some weights of BartForSequenceClassification were not initialized from the model checkpoint at facebook/bart-base and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Predicted Label Distribution: Counter({1: 1643, 0: 133})
True Label Distribution: Counter({1: 888, 0: 888})

--- Evaluation Examples by Relation Type ---

Relation: Continuation
Input: no thanks <SEP> sheep?
Gold: 1 | Pred: 1

Input: no thanks <SEP> anyone need ore?
Gold: 0 | Pred: 1

Input: no thanks <EDU_SEP> sheep? <EDU_SEP> yes, <EDU_SEP> i want sheep:) <EDU_SEP> me too <EDU_SEP> me too <EDU_SEP> i want them too <EDU_SEP> lol <EDU_SEP> but i am trading... <EDU_SEP> robber <EDU_SEP> why put it on sheep? <EDU_SEP> it's literally at random where i put it <EDU_SEP> sorry <SEP> its no good!
Gold: 1 | Pred: 1


Relation: Question_answer_pair
Input: no thanks <EDU_SEP> sheep? <SEP> yes,
Gold: 1 | Pred: 1

Input: no thanks <EDU_SEP> sheep? <SEP> uh oh
Gold: 0 | Pred: 1

Input: no thanks <EDU_SEP> sheep? <EDU_SEP> yes, <EDU_SEP> i want sheep:) <EDU_SEP> me too <SEP> me too
Gold: 1 | Pred: 1


Relation: Elaboration
Input: no thanks <EDU_SEP> sheep? <EDU_SEP> yes, <SEP> i want sheep:)
Gold: 1

## BART NSP (significance)

In [ ]:
# Export BART NSP BASELINE per-dialogue CSV
import os, pandas as pd

BASELINE_OUT = "/content/exports/bart_nsp_baseline.csv"
FINETUNED_IN = "/content/exports/bart_nsp_finetuned.csv"  # for consistent IDs/relations
os.makedirs("/content/exports", exist_ok=True)

# Must exist from the baseline eval cell:
assert 'bart_nsp_acc_baseline' in globals() and len(bart_nsp_acc_baseline) > 0, \
    "Missing 'bart_nsp_acc_baseline' — run the BASELINE NSP eval cell first."

# Prefer to reuse finetuned IDs for perfect pairing
assert os.path.exists(FINETUNED_IN), f"Finetuned CSV not found at {FINETUNED_IN}. Run N1 (finetuned export) first."
fin = pd.read_csv(FINETUNED_IN)

N = min(len(fin), len(bart_nsp_acc_baseline))
df_base = pd.DataFrame({
    "dialogue_id":   fin["dialogue_id"].iloc[:N].astype(str).tolist(),
    "relation_type": fin["relation_type"].iloc[:N].astype(str).tolist(),
    "metric_value":  [float(x) for x in bart_nsp_acc_baseline[:N]],  # accuracy in [0,1]
})

df_base.to_csv(BASELINE_OUT, index=False)
print(f"Saved → {BASELINE_OUT}  rows: {len(df_base)}")
display(df_base.head(5))

Saved → /content/exports/bart_nsp_baseline.csv  rows: 1776


,dialogue_id,relation_type,metric_value
0,dlg_auto_000000,Question_answer_pair,1.0
1,dlg_auto_000001,Question_answer_pair,0.0
2,dlg_auto_000002,Q_Elab,1.0
3,dlg_auto_000003,Q_Elab,0.0
4,dlg_auto_000004,Question_answer_pair,1.0


In [ ]:
# BART NSP significance (paired Wilcoxon + bootstrap CI + per-relation)
import os, numpy as np, pandas as pd
from scipy.stats import wilcoxon

BASELINE   = "/content/exports/bart_nsp_baseline.csv"
FINETUNED  = "/content/exports/bart_nsp_finetuned.csv"
OUT_PREFIX = "/content/exports/bart_nsp"
MIN_N      = 8
METRIC     = "acc"
CHANCE     = 0.5

def _bootstrap_mean_diff_CI(x, y, n_boot=10000, seed=42, alpha=0.05):
    rng = np.random.default_rng(seed)
    diffs = np.asarray(x) - np.asarray(y)
    n = len(diffs)
    idx = rng.integers(0, n, size=(n_boot, n))
    boot_means = diffs[idx].mean(axis=1)
    return float(np.quantile(boot_means, alpha/2)), float(np.quantile(boot_means, 1 - alpha/2))

# Load & merge
assert os.path.exists(BASELINE), f"Missing: {BASELINE}"
assert os.path.exists(FINETUNED), f"Missing: {FINETUNED}"
b = pd.read_csv(BASELINE)
f = pd.read_csv(FINETUNED)
for req in ["dialogue_id","relation_type","metric_value"]:
    assert req in b.columns and req in f.columns, f"Missing '{req}' in one of the CSVs."

df = pd.merge(b, f, on="dialogue_id", suffixes=("_baseline","_finetuned"))
df["relation_type"] = df.get("relation_type_finetuned", df.get("relation_type_baseline","Unknown"))
df = df[["dialogue_id","relation_type","metric_value_baseline","metric_value_finetuned"]].dropna()
assert len(df) > 0, "No overlapping dialogues after merge."

if np.allclose(df["metric_value_baseline"], df["metric_value_finetuned"]):
    raise ValueError("Baseline and finetuned arrays are identical — check your NSP exports.")

# Overall tests
x = df["metric_value_finetuned"].to_numpy()
y = df["metric_value_baseline"].to_numpy()
w_stat, w_p = wilcoxon(x, y, zero_method="wilcox", alternative="two-sided")
w_stat_ch, w_p_ch = wilcoxon(x, np.full_like(x, CHANCE, dtype=float), zero_method="wilcox", alternative="greater")
ci_lo, ci_hi = _bootstrap_mean_diff_CI(x, y)

overall = pd.DataFrame([{
    "n_dialogues": len(df),
    f"baseline_mean_{METRIC}": float(y.mean()),
    f"finetuned_mean_{METRIC}": float(x.mean()),
    "mean_diff": float(x.mean() - y.mean()),
    "boot_CI_low": ci_lo, "boot_CI_high": ci_hi,
    "wilcoxon_stat": float(w_stat), "wilcoxon_p": float(w_p),
    f"chance_test_p(finetuned>{CHANCE})": float(w_p_ch),
}])

# By-relation tests
rows = []
for rel, grp in df.groupby("relation_type"):
    n = len(grp)
    base = grp["metric_value_baseline"].to_numpy()
    fine = grp["metric_value_finetuned"].to_numpy()
    mean_b, mean_f = float(base.mean()), float(fine.mean())
    diff = float(mean_f - mean_b)
    if n >= MIN_N:
        ws, wp = wilcoxon(fine, base, zero_method="wilcox", alternative="two-sided")
        ws_c, wp_c = wilcoxon(fine, np.full_like(fine, CHANCE, dtype=float), zero_method="wilcox", alternative="greater")
        p_pair, p_chance = float(wp), float(wp_c)
    else:
        p_pair = p_chance = ""
    rows.append({
        "relation_type": rel, "n": int(n),
        f"baseline_mean_{METRIC}": mean_b,
        f"finetuned_mean_{METRIC}": mean_f,
        "mean_diff": diff,
        "wilcoxon_p(baseline_vs_finetuned)": p_pair,
        f"chance_test_p(finetuned>{CHANCE})": p_chance
    })
byrel = pd.DataFrame(rows).sort_values(["n","relation_type"], ascending=[False, True])

# Save + display
os.makedirs(os.path.dirname(OUT_PREFIX), exist_ok=True)
overall.to_csv(f"{OUT_PREFIX}_significance_overall.csv", index=False)
byrel.to_csv(f"{OUT_PREFIX}_significance_by_relation.csv", index=False)
display(overall); display(byrel)

r = overall.iloc[0]
print(
  f"Significance (BART NSP). The improvement from baseline ({METRIC} = {r[f'baseline_mean_{METRIC}']:.4f}) "
  f"to fine-tuned ({METRIC} = {r[f'finetuned_mean_{METRIC}']:.4f}) was statistically significant "
  f"(Wilcoxon signed-rank, p = {r['wilcoxon_p']:.3g}; 95% CI for mean difference: "
  f"[{r['boot_CI_low']:.3f}, {r['boot_CI_high']:.3f}]). Fine-tuned performance was also significantly above chance "
  f"({METRIC} > {CHANCE}; p = {r[f'chance_test_p(finetuned>{CHANCE})']:.3g})."
)

,n_dialogues,baseline_mean_acc,finetuned_mean_acc,mean_diff,boot_CI_low,boot_CI_high,wilcoxon_stat,wilcoxon_p,chance_test_p(finetuned>0.5)
0,1776,0.499437,0.636824,0.137387,0.10473,0.170608,147764.0,4.176442e-16,4.536001e-31


,relation_type,n,baseline_mean_acc,finetuned_mean_acc,mean_diff,wilcoxon_p(baseline_vs_finetuned),chance_test_p(finetuned>0.5)
14,Question_answer_pair,382,0.497382,0.675393,0.178010,8.087730e-07,3.539866e-12
4,Comment,364,0.502747,0.543956,0.041209,2.777614e-01,4.674624e-02
0,Acknowledgement,260,0.500000,0.646154,0.146154,1.028180e-03,1.218566e-06
6,Continuation,174,0.494253,0.678161,0.183908,5.592409e-04,1.299484e-06
9,Elaboration,104,0.509615,0.548077,0.038462,5.553463e-01,1.633998e-01
13,Q_Elab,88,0.500000,0.681818,0.181818,1.355467e-02,3.233746e-04
3,Clarification_question,80,0.500000,0.725000,0.225000,7.955439e-03,2.849706e-05
10,Explanation,80,0.500000,0.675000,0.175000,3.075356e-02,8.725593e-04
7,Contrast,64,0.484375,0.671875,0.187500,5.157586e-02,2.979763e-03
15,Result,52,0.500000,0.730769,0.230769,7.290358e-03,4.370436e-04


Significance (BART NSP). The improvement from baseline (acc = 0.4994) to fine-tuned (acc = 0.6368) was statistically significant (Wilcoxon signed-rank, p = 4.18e-16; 95% CI for mean difference: [0.105, 0.171]). Fine-tuned performance was also significantly above chance (acc > 0.5; p = 4.54e-31).


# RoBERTa NSP Preprocessing



In [ ]:
import json
import random
import pandas as pd
from collections import Counter
from google.colab import files

# Load and clean the dataset
df = pd.read_csv("incoming_base_spect.csv")
df = df[df['relation_type'].notna() & (df['relation_type'] != "0")]

# Group by dialogue
grouped = df.groupby(['doc', 'dialogue_num'])

examples = []

# Process each dialogue
for (doc_id, dlg_num), group in grouped:
    edus = group['text'].dropna().tolist()
    relations = group['relation_type'].dropna().tolist()

    if not (2 <= len(edus) <= 10):
        continue

    dialogue_id = f"{doc_id}_{dlg_num}"
    num_edus = len(edus)

    for i in range(2, min(num_edus, 9)):  # i is the index of the target EDU
        context = edus[:i]
        target = edus[i]
        rel_type = relations[i - 1]  # relation between last context EDU and target

        context_str = " [EDU_SEP] ".join(context)

        # Positive example
        examples.append({
            "dialogue_id": dialogue_id,
            "input": context_str + " [CTX_SEP] " + target,
            "label": 1,
            "relation_type": rel_type
        })

        # Negative example
        neg_candidates = [j for j in range(num_edus) if j != i and abs(j - i) > 1]
        if not neg_candidates:
            continue

        j = random.choice(neg_candidates)
        negative_target = edus[j]

        examples.append({
            "dialogue_id": dialogue_id,
            "input": context_str + " [CTX_SEP] " + negative_target,
            "label": 0,
            "relation_type": rel_type
        })

# Save all NSP examples
full_path = "roberta_nsp.jsonl"
with open(full_path, "w", encoding="utf-8") as f:
    for item in examples:
        json.dump(item, f, ensure_ascii=False)
        f.write("\n")

print(f"Saved {len(examples)} NSP examples → {full_path}")
files.download(full_path)

# Split into Train/Dev/Test
def split_json_data(data, output_prefix, seed=42, train_ratio=0.8, dev_ratio=0.1):
    random.seed(seed)
    random.shuffle(data)

    total = len(data)
    train_end = int(train_ratio * total)
    dev_end = train_end + int(dev_ratio * total)

    splits = {
        "train": data[:train_end],
        "dev": data[train_end:dev_end],
        "test": data[dev_end:]
    }

    for split_name, split_data in splits.items():
        path = f"{output_prefix}_{split_name}.jsonl"
        with open(path, "w", encoding="utf-8") as f:
            for item in split_data:
                json.dump(item, f, ensure_ascii=False)
                f.write("\n")
        print(f"Saved {len(split_data)} → {path}")
        files.download(path)

# Run the split
split_json_data(examples, "roberta_nsp")

Saved 4520 NSP examples → roberta_nsp.jsonl


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Saved 3616 → roberta_nsp_train.jsonl


<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Saved 452 → roberta_nsp_test.jsonl


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# RoBERTa NSP Fine-tuning

In [ ]:
import json, os, random
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer, RobertaForSequenceClassification, get_linear_schedule_with_warmup
from torch.optim import AdamW
from torch.nn.utils import clip_grad_norm_
from sklearn.metrics import accuracy_score
from tqdm import tqdm

# Parameters
train_file = "roberta_nsp_train.jsonl"
dev_file   = "roberta_nsp_dev.jsonl"
batch_size = 8
lr         = 2e-5
epochs     = 5
max_length = 512
warmup_pct = 0.10
weight_decay = 0.01
grad_clip = 1.0
seed = 42

# Reproducibility
random.seed(seed); np.random.seed(seed)
torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)

# Load tokenizer & model
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
tokenizer.add_special_tokens({"additional_special_tokens": ["[CTX_SEP]", "[EDU_SEP]"]})
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=2)
model.resize_token_embeddings(len(tokenizer))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Utilities to read labels for class weights
def load_labels(path):
    with open(path, "r", encoding="utf-8") as f:
        return [int(json.loads(line)["label"]) for line in f]

train_labels = load_labels(train_file)
# Class weights (inverse frequency): w_c = N / (2 * count_c)
counts = np.bincount(train_labels, minlength=2).astype(float)
counts[counts == 0] = 1.0
N = counts.sum()
class_weights = torch.tensor([N/(2*counts[0]), N/(2*counts[1])], dtype=torch.float)

# Dataset class (pair-encode if [CTX_SEP] is present)
class RobertaNSPDataset(Dataset):
    def __init__(self, path, tokenizer):
        with open(path, "r", encoding="utf-8") as f:
            self.data = [json.loads(line) for line in f]
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        text = item["input"]
        label = int(item["label"])

        if "[CTX_SEP]" in text:
            a, b = text.split("[CTX_SEP]", 1)
            enc = self.tokenizer(
                a.strip(), b.strip(),
                truncation=True,
                padding="max_length",
                max_length=max_length,
                return_tensors="pt"
            )
        else:
            enc = self.tokenizer(
                text,
                truncation=True,
                padding="max_length",
                max_length=max_length,
                return_tensors="pt"
            )

        return {
            "input_ids": enc["input_ids"].squeeze(0),
            "attention_mask": enc["attention_mask"].squeeze(0),
            "labels": torch.tensor(label, dtype=torch.long)
        }

# Load data
train_dataset = RobertaNSPDataset(train_file, tokenizer)
dev_dataset   = RobertaNSPDataset(dev_file, tokenizer)
train_loader  = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
dev_loader    = DataLoader(dev_dataset,   batch_size=batch_size, shuffle=False)

# Optimizer, scheduler, loss
optimizer = AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
total_steps = epochs * max(1, len(train_loader))
warmup_steps = int(warmup_pct * total_steps)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps)
ce_loss = torch.nn.CrossEntropyLoss(weight=class_weights.to(device))

best_dev_acc = -1.0
best_epoch = -1

for epoch in range(1, epochs + 1):
    # Train
    model.train()
    total_loss = 0.0
    for batch in tqdm(train_loader, desc=f"Epoch {epoch}"):
        batch = {k: v.to(device) for k, v in batch.items()}
        out = model(input_ids=batch["input_ids"], attention_mask=batch["attention_mask"])
        logits = out.logits
        loss = ce_loss(logits, batch["labels"])

        loss.backward()
        clip_grad_norm_(model.parameters(), grad_clip)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad(set_to_none=True)

        total_loss += loss.item()

    avg_train_loss = total_loss / max(1, len(train_loader))
    print(f"Epoch {epoch}: Training Loss = {avg_train_loss:.4f}")

    # Validate (report accuracy; select best by accuracy)
    model.eval()
    dev_loss = 0.0
    preds_all, gold_all = [], []
    with torch.no_grad():
        for batch in dev_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            out = model(input_ids=batch["input_ids"], attention_mask=batch["attention_mask"])
            logits = out.logits
            loss = ce_loss(logits, batch["labels"])
            dev_loss += loss.item()

            probs = torch.softmax(logits, dim=-1)[:, 1]
            preds = (probs >= 0.5).long()   # keep 0.5; you can calibrate at eval time
            preds_all.extend(preds.cpu().tolist())
            gold_all.extend(batch["labels"].cpu().tolist())

    avg_dev_loss = dev_loss / max(1, len(dev_loader))
    dev_acc = accuracy_score(gold_all, preds_all)
    pos_rate_train = np.mean(train_labels)
    pos_rate_dev   = np.mean(gold_all) if len(gold_all) > 0 else float("nan")
    print(f"Epoch {epoch}: Dev Loss = {avg_dev_loss:.4f} | Dev Acc = {dev_acc:.4f} | "
          f"PosRate (train/dev) = {pos_rate_train:.3f}/{pos_rate_dev:.3f}")

    # Save best by dev accuracy
    if dev_acc > best_dev_acc:
        best_dev_acc = dev_acc
        best_epoch = epoch
        # Save state_dict (legacy)
        torch.save(model.state_dict(), "roberta_nsp_best.pt")
        # Also save a full directory for from_pretrained()
        save_dir = "roberta_nsp_best"
        os.makedirs(save_dir, exist_ok=True)
        tokenizer.save_pretrained(save_dir)
        model.save_pretrained(save_dir)
        print(f"Best model saved at epoch {epoch} → acc={best_dev_acc:.4f}")

print(f"Training complete. Best epoch: {best_epoch} (Dev Acc = {best_dev_acc:.4f})")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

...

Epoch 3: Training Loss = 0.4286
Epoch 3: Dev Loss = 0.5306 | Dev Acc = 0.8053 | PosRate (train/dev) = 0.497/0.502


Epoch 4: 100%|██████████| 452/452 [05:32<00:00,  1.36it/s]


Epoch 4: Training Loss = 0.3835
Epoch 4: Dev Loss = 0.5822 | Dev Acc = 0.8186 | PosRate (train/dev) = 0.497/0.502


Epoch 5: 100%|██████████| 452/452 [05:32<00:00,  1.36it/s]


Epoch 5: Training Loss = 0.3702
Epoch 5: Dev Loss = 0.5579 | Dev Acc = 0.8208 | PosRate (train/dev) = 0.497/0.502
Training complete. Best epoch: 2 (Dev Acc = 0.8274)


# Fine-tuned RoBERTa NSP Evaluation

In [ ]:
import os
import json
import torch
import numpy as np
import pandas as pd
from collections import defaultdict, Counter
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from sklearn.metrics import f1_score, accuracy_score

# Parameters
test_file     = "roberta_nsp_test.jsonl"
batch_size    = 8
max_length    = 512
finetuned_dir = "roberta_nsp_best"
save_csv_path = "/content/exports/roberta_nsp_finetuned_dialogue_eval.csv"

# Load tokenizer and model (fine-tuned)
assert os.path.isdir(finetuned_dir), (
    f"Missing '{finetuned_dir}' directory. Save the fine-tuned model with save_pretrained(save_dir)."
)
tokenizer = RobertaTokenizer.from_pretrained(finetuned_dir)
model = RobertaForSequenceClassification.from_pretrained(finetuned_dir)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device).eval()

# Dataset
class RobertaNSPDataset(Dataset):
    def __init__(self, path, tokenizer):
        with open(path, "r", encoding="utf-8") as f:
            self.data = [json.loads(line) for line in f]
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item  = self.data[idx]
        text  = item["input"]
        label = int(item["label"])
        rel   = item.get("relation") or item.get("relation_type") or "Unknown"
        dlgid = (item.get("dialogue_id") or item.get("dlg_id")
                 or item.get("dialogue") or item.get("doc_id")
                 or item.get("doc") or "UNKNOWN")

        if "[CTX_SEP]" in text:
            a, b = text.split("[CTX_SEP]", 1)
            enc = self.tokenizer(
                a.strip(), b.strip(),
                truncation=True,
                padding="max_length",
                max_length=max_length,
                return_tensors="pt"
            )
        else:
            enc = self.tokenizer(
                text,
                truncation=True,
                padding="max_length",
                max_length=max_length,
                return_tensors="pt"
            )

        return {
            "input_ids": enc["input_ids"].squeeze(),
            "attention_mask": enc["attention_mask"].squeeze(),
            "labels": torch.tensor(label, dtype=torch.long),
            "relation": rel,
            "input": text,
            "dialogue_id": dlgid
        }

# Load test data
test_dataset = RobertaNSPDataset(test_file, tokenizer)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Collect predictions (pair-level) → aggregate to dialogue-level
all_dialogue_ids, all_labels, all_preds, all_rels, all_inputs = [], [], [], [], []

with torch.no_grad():
    for batch in test_loader:
        input_ids      = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels         = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        preds = torch.argmax(outputs.logits, dim=1).cpu().numpy()
        golds = labels.cpu().numpy()

        all_dialogue_ids.extend(batch["dialogue_id"])
        all_rels.extend(batch["relation"])
        all_labels.extend(golds.tolist())
        all_preds.extend(preds.tolist())
        all_inputs.extend(batch["input"])  # collect full inputs for printing

# Aggregate by dialogue & collect examples per relation
dlg_bins = defaultdict(lambda: {"labels": [], "preds": [], "rels": []})
examples_by_rel = defaultdict(list)

for dlgid, y_true, y_hat, rel, inp in zip(all_dialogue_ids, all_labels, all_preds, all_rels, all_inputs):
    dlg_bins[dlgid]["labels"].append(y_true)
    dlg_bins[dlgid]["preds"].append(y_hat)
    dlg_bins[dlgid]["rels"].append(rel)
    examples_by_rel[rel].append((inp, y_true, y_hat))

def balanced_accuracy_01(y_true, y_pred):
    """Macro recall over {0,1}; if a class is absent in this dialogue, treat that class-recall as 0.5 (chance)."""
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    parts = []
    for cls in (0, 1):
        mask = (y_true == cls)
        parts.append(0.5 if mask.sum() == 0 else float((y_pred[mask] == cls).mean()))
    return float(np.mean(parts))

def macro_f1_or_nan(y_true, y_pred):
    if len(set(y_true)) < 2:
        return np.nan
    try:
        return f1_score(y_true, y_pred, average="macro")
    except Exception:
        return np.nan

rows = []
for dlgid, d in dlg_bins.items():
    labels = d["labels"]
    preds  = d["preds"]
    rels   = d["rels"]

    acc_plain = accuracy_score(labels, preds)
    acc_bal   = balanced_accuracy_01(labels, preds)
    mf1       = macro_f1_or_nan(labels, preds)
    dominant_rel = Counter(rels).most_common(1)[0][0] if len(rels) else "Unknown"

    rows.append({
        "dialogue_id": dlgid,
        "n_pairs": len(labels),
        "acc_plain": acc_plain,
        "acc_balanced": acc_bal,
        "macro_f1": mf1,
        "dominant_relation": dominant_rel
    })

dlg_df = pd.DataFrame(rows)
n_dialogues = len(dlg_df)

# Headline metrics (BALANCED to match significance)
headline_mean_bal = dlg_df["acc_balanced"].mean()
mean_macro_f1     = dlg_df["macro_f1"].dropna().mean() if dlg_df["macro_f1"].notna().any() else np.nan

print("\n=== RoBERTa NSP (Fine-tuned) — Dialogue-level (BALANCED) Evaluation ===")
print(f"Dialogues: {n_dialogues}")
print(f"Per-dialogue mean (BALANCED) accuracy: {headline_mean_bal:.6f}")
print("Per-dialogue mean macro-F1:  n/a" if np.isnan(mean_macro_f1) else f"Per-dialogue mean macro-F1:  {mean_macro_f1:.6f}")

# Dialogue-level by dominant relation (using BALANCED accuracy)
by_rel = (dlg_df.groupby("dominant_relation", dropna=False)
          .agg(n_dialogues=("acc_balanced", "count"),
               mean_bal_acc=("acc_balanced", "mean"),
               mean_macro_f1=("macro_f1", "mean"))
          .reset_index()
          .sort_values(["n_dialogues", "mean_bal_acc"], ascending=[False, False]))

print("\n--- Dialogue-level (BALANCED) Evaluation by Dominant Discourse Relation ---")
for _, r in by_rel.iterrows():
    rel = r["dominant_relation"]
    n   = int(r["n_dialogues"])
    ma  = r["mean_bal_acc"]
    mf1 = r["mean_macro_f1"]
    mf1_str = "(skip: undefined)" if np.isnan(mf1) else f"{mf1:.3f}"
    print(f"{str(rel):<24} | n_dialogues={n:>3} | mean Bal. Acc: {ma:>6.3f} | mean Macro F1: {mf1_str}")

# Print ALL examples grouped by relation
print("\n--- All Examples by Relation ---")
for rel, examples in examples_by_rel.items():
    print(f"\nRelation: {rel} (n={len(examples)})")
    for inp, gold, pred in examples:
        print(f"Input: {inp}\nGold: {gold} | Pred: {pred}\n")

# Save CSV aligned with significance cell (metric_value = balanced acc)
os.makedirs(os.path.dirname(save_csv_path), exist_ok=True)
out = dlg_df.copy()
out["metric_value"] = out["acc_balanced"]
out.to_csv(save_csv_path, index=False)
print(f"\nSaved per-dialogue results → {save_csv_path}")


=== RoBERTa NSP (Fine-tuned) — Dialogue-level (BALANCED) Evaluation ===
Dialogues: 293
Per-dialogue mean (BALANCED) accuracy: 0.696132
Per-dialogue mean macro-F1:  0.778767

--- Dialogue-level (BALANCED) Evaluation by Dominant Discourse Relation ---
Question_answer_pair     | n_dialogues= 88 | mean Bal. Acc:  0.690 | mean Macro F1: 0.786
Comment                  | n_dialogues= 52 | mean Bal. Acc:  0.708 | mean Macro F1: 0.840
Acknowledgement          | n_dialogues= 37 | mean Bal. Acc:  0.691 | mean Macro F1: 0.781
Elaboration              | n_dialogues= 21 | mean Bal. Acc:  0.762 | mean Macro F1: 1.000
Q_Elab                   | n_dialogues= 20 | mean Bal. Acc:  0.654 | mean Macro F1: 0.680
Continuation             | n_dialogues= 18 | mean Bal. Acc:  0.708 | mean Macro F1: 0.813
Explanation              | n_dialogues= 15 | mean Bal. Acc:  0.717 | mean Macro F1: 0.889
Contrast                 | n_dialogues= 14 | mean Bal. Acc:  0.643 | mean Macro F1: 0.333
Clarification_question   | n_

In [ ]:
# Export RoBERTa NSP FINETUNED per-dialogue CSV
import os, pandas as pd

OUT_PATH = "/content/exports/roberta_nsp_finetuned.csv"
os.makedirs("/content/exports", exist_ok=True)

# Expect from eval cell:
assert 'roberta_nsp_acc_finetuned' in globals() and len(roberta_nsp_acc_finetuned) > 0, \
    "Missing 'roberta_nsp_acc_finetuned' — run the RoBERTa NSP finetuned eval first."

N = len(roberta_nsp_acc_finetuned)
ids  = globals().get('roberta_nsp_dialogue_ids_fine', [f"dlg_{i:04d}" for i in range(N)])
rels = globals().get('roberta_nsp_relations_fine',   ["Unknown"] * N)

N = min(N, len(ids), len(rels))
df_fin = pd.DataFrame({
    "dialogue_id":   [str(x) for x in ids[:N]],
    "relation_type": [str(x) for x in rels[:N]],
    "metric_value":  [float(x) for x in roberta_nsp_acc_finetuned[:N]],  # accuracy in [0,1]
})

df_fin.to_csv(OUT_PATH, index=False)
print(f"Saved → {OUT_PATH}  rows: {len(df_fin)}")
display(df_fin.head(5))

Saved → /content/exports/roberta_nsp_finetuned.csv  rows: 293


,dialogue_id,relation_type,metric_value
0,pilot01_12.0,Explanation,0.0
1,pilot01_13.0,Elaboration,0.5
2,pilot01_19.0,Q_Elab,0.5
3,pilot01_2.0,Contrast,0.5
4,pilot01_23.0,Acknowledgement,1.0


# RoBERTa NSP Baseline Evaluation

In [ ]:
import os
import json
import torch
import numpy as np
import pandas as pd
from collections import defaultdict, Counter
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from sklearn.metrics import f1_score, accuracy_score

# Parameters
test_file       = "roberta_nsp_test.jsonl"
batch_size      = 8
max_length      = 512
baseline_model  = "roberta-base"  # pretrained baseline (no fine-tuning)
save_csv_path   = "/content/exports/roberta_nsp_baseline_dialogue_eval.csv"

# Load tokenizer and model (baseline)
tokenizer = RobertaTokenizer.from_pretrained(baseline_model)
model     = RobertaForSequenceClassification.from_pretrained(baseline_model, num_labels=2)
device    = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device).eval()

# Dataset
class RobertaNSPDataset(Dataset):
    def __init__(self, path, tokenizer):
        with open(path, "r", encoding="utf-8") as f:
            self.data = [json.loads(line) for line in f]
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item  = self.data[idx]
        text  = item["input"]
        label = int(item["label"])
        rel   = item.get("relation") or item.get("relation_type") or "Unknown"
        dlgid = (item.get("dialogue_id") or item.get("dlg_id")
                 or item.get("dialogue") or item.get("doc_id")
                 or item.get("doc") or "UNKNOWN")

        if "[CTX_SEP]" in text:
            a, b = text.split("[CTX_SEP]", 1)
            enc = self.tokenizer(
                a.strip(), b.strip(),
                truncation=True, padding="max_length", max_length=max_length,
                return_tensors="pt"
            )
        else:
            enc = self.tokenizer(
                text,
                truncation=True, padding="max_length", max_length=max_length,
                return_tensors="pt"
            )

        return {
            "input_ids": enc["input_ids"].squeeze(),
            "attention_mask": enc["attention_mask"].squeeze(),
            "labels": torch.tensor(label, dtype=torch.long),
            "relation": rel,
            "input": text,
            "dialogue_id": dlgid
        }

# Load test data
test_dataset = RobertaNSPDataset(test_file, tokenizer)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Collect predictions (pair-level) → aggregate to dialogue-level
all_dialogue_ids, all_labels, all_preds, all_rels, all_inputs = [], [], [], [], []

with torch.no_grad():
    for batch in test_loader:
        input_ids      = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels         = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        preds = torch.argmax(outputs.logits, dim=1).cpu().numpy()
        golds = labels.cpu().numpy()

        all_dialogue_ids.extend(batch["dialogue_id"])
        all_rels.extend(batch["relation"])
        all_labels.extend(golds.tolist())
        all_preds.extend(preds.tolist())
        all_inputs.extend(batch["input"])

# Aggregate by dialogue & collect examples per relation
dlg_bins = defaultdict(lambda: {"labels": [], "preds": [], "rels": []})
examples_by_rel = defaultdict(list)

for dlgid, y_true, y_hat, rel, inp in zip(all_dialogue_ids, all_labels, all_preds, all_rels, all_inputs):
    dlg_bins[dlgid]["labels"].append(y_true)
    dlg_bins[dlgid]["preds"].append(y_hat)
    dlg_bins[dlgid]["rels"].append(rel)
    examples_by_rel[rel].append((inp, y_true, y_hat))

def balanced_accuracy_01(y_true, y_pred):
    """Macro recall over {0,1}; if a class is absent in this dialogue, treat that class-recall as 0.5 (chance)."""
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    parts = []
    for cls in (0, 1):
        mask = (y_true == cls)
        parts.append(0.5 if mask.sum() == 0 else float((y_pred[mask] == cls).mean()))
    return float(np.mean(parts))

def macro_f1_or_nan(y_true, y_pred):
    if len(set(y_true)) < 2:
        return np.nan
    try:
        return f1_score(y_true, y_pred, average="macro")
    except Exception:
        return np.nan

rows = []
for dlgid, d in dlg_bins.items():
    labels = d["labels"]
    preds  = d["preds"]
    rels   = d["rels"]

    acc_plain = accuracy_score(labels, preds)
    acc_bal   = balanced_accuracy_01(labels, preds)
    mf1       = macro_f1_or_nan(labels, preds)
    dominant_rel = Counter(rels).most_common(1)[0][0] if len(rels) else "Unknown"

    rows.append({
        "dialogue_id": dlgid,
        "n_pairs": len(labels),
        "acc_plain": acc_plain,
        "acc_balanced": acc_bal,
        "macro_f1": mf1,
        "dominant_relation": dominant_rel
    })

dlg_df = pd.DataFrame(rows)
n_dialogues = len(dlg_df)

# Headline metrics (BALANCED to match significance)
headline_mean_bal = dlg_df["acc_balanced"].mean()
mean_macro_f1     = dlg_df["macro_f1"].dropna().mean() if dlg_df["macro_f1"].notna().any() else np.nan

print("\n=== RoBERTa NSP (Baseline) — Dialogue-level (BALANCED) Evaluation ===")
print(f"Dialogues: {n_dialogues}")
print(f"Per-dialogue mean (BALANCED) accuracy: {headline_mean_bal:.6f}")
print("Per-dialogue mean macro-F1:  n/a" if np.isnan(mean_macro_f1) else f"Per-dialogue mean macro-F1:  {mean_macro_f1:.6f}")

print("\n--- Dialogue-level (BALANCED) Evaluation by Dominant Discourse Relation ---")
by_rel = (dlg_df.groupby("dominant_relation", dropna=False)
          .agg(n_dialogues=("acc_balanced", "count"),
               mean_bal_acc=("acc_balanced", "mean"),
               mean_macro_f1=("macro_f1", "mean"))
          .reset_index()
          .sort_values(["n_dialogues", "mean_bal_acc"], ascending=[False, False]))
for _, r in by_rel.iterrows():
    rel = r["dominant_relation"]
    n   = int(r["n_dialogues"])
    ma  = r["mean_bal_acc"]
    mf1 = r["mean_macro_f1"]
    mf1_str = "(skip: undefined)" if np.isnan(mf1) else f"{mf1:.3f}"
    print(f"{str(rel):<24} | n_dialogues={n:>3} | mean Bal. Acc: {ma:>6.3f} | mean Macro F1: {mf1_str}")

# Print ALL examples grouped by relation
print("\n--- All Examples by Relation ---")
for rel, examples in examples_by_rel.items():
    print(f"\nRelation: {rel} (n={len(examples)})")
    for inp, gold, pred in examples:
        print(f"Input: {inp}\nGold: {gold} | Pred: {pred}\n")

# Save CSV aligned with significance cell (metric_value = balanced acc)
os.makedirs(os.path.dirname(save_csv_path), exist_ok=True)
out = dlg_df.copy()
out["metric_value"] = out["acc_balanced"]
out.to_csv(save_csv_path, index=False)
print(f"\nSaved per-dialogue results → {save_csv_path}")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



=== RoBERTa NSP (Baseline) — Dialogue-level (BALANCED) Evaluation ===
Dialogues: 293
Per-dialogue mean (BALANCED) accuracy: 0.489761
Per-dialogue mean macro-F1:  0.326158

--- Dialogue-level (BALANCED) Evaluation by Dominant Discourse Relation ---
Question_answer_pair     | n_dialogues= 88 | mean Bal. Acc:  0.491 | mean Macro F1: 0.329
Comment                  | n_dialogues= 52 | mean Bal. Acc:  0.500 | mean Macro F1: 0.317
Acknowledgement          | n_dialogues= 37 | mean Bal. Acc:  0.520 | mean Macro F1: 0.331
Elaboration              | n_dialogues= 21 | mean Bal. Acc:  0.440 | mean Macro F1: 0.333
Q_Elab                   | n_dialogues= 20 | mean Bal. Acc:  0.425 | mean Macro F1: 0.320
Continuation             | n_dialogues= 18 | mean Bal. Acc:  0.431 | mean Macro F1: 0.336
Explanation              | n_dialogues= 15 | mean Bal. Acc:  0.533 | mean Macro F1: 0.328
Contrast                 | n_dialogues= 14 | mean Bal. Acc:  0.464 | mean Macro F1: 0.333
Clarification_question   | n_di

In [ ]:
# Export RoBERTa NSP BASELINE per-dialogue CSV (for paired significance tests)
import os, pandas as pd

BASELINE_OUT = "/content/exports/roberta_nsp_baseline.csv"
FINETUNED_IN = "/content/exports/roberta_nsp_finetuned.csv"
os.makedirs("/content/exports", exist_ok=True)

assert 'roberta_nsp_acc_baseline' in globals() and len(roberta_nsp_acc_baseline) > 0, \
    "Missing 'roberta_nsp_acc_baseline' — run the RoBERTa NSP baseline eval cell first."

assert os.path.exists(FINETUNED_IN), \
    f"Finetuned CSV not found at {FINETUNED_IN}. Run the finetuned export cell (E2) first."
fin = pd.read_csv(FINETUNED_IN)

N = min(len(fin), len(roberta_nsp_acc_baseline))
df_base = pd.DataFrame({
    "dialogue_id":   fin["dialogue_id"].iloc[:N].astype(str).tolist(),
    "relation_type": fin["relation_type"].iloc[:N].astype(str).tolist(),
    "metric_value":  [float(x) for x in roberta_nsp_acc_baseline[:N]],  # accuracy in [0,1]
})

df_base.to_csv(BASELINE_OUT, index=False)
print(f"Saved → {BASELINE_OUT}  rows: {len(df_base)}  (paired with finetuned IDs)")
display(df_base.head(5))


Saved → /content/exports/roberta_nsp_baseline.csv  rows: 293  (paired with finetuned IDs)


,dialogue_id,relation_type,metric_value
0,pilot01_12.0,Explanation,0.0
1,pilot01_13.0,Elaboration,0.0
2,pilot01_19.0,Q_Elab,0.0
3,pilot01_2.0,Contrast,0.5
4,pilot01_23.0,Acknowledgement,0.0


## RoBERTa NSP (significance)

In [ ]:
# RoBERTa NSP Significance Test (Balanced Accuracy)
import pandas as pd
import numpy as np
from scipy.stats import wilcoxon

def binom_p_greater(k, n, p=0.5):
    try:
        from scipy.stats import binomtest  # SciPy >=1.7
        return binomtest(k, n, p, alternative="greater").pvalue
    except Exception:
        try:
            from scipy.stats import binom
            return float(binom.sf(k-1, n, p))  # P[X >= k]
        except Exception:
            from math import comb
            s = 0.0
            for i in range(k, n+1):
                s += comb(n, i) * (p**i) * ((1-p)**(n-i))
            return float(s)

# Load dialogue-level results (balanced accuracy CSVs saved by your eval cells)
base_csv = "/content/exports/roberta_nsp_baseline_dialogue_eval.csv"
ft_csv   = "/content/exports/roberta_nsp_finetuned_dialogue_eval.csv"

df_base = pd.read_csv(base_csv)
df_ft   = pd.read_csv(ft_csv)

# Align by dialogue_id
merged = pd.merge(
    df_base[["dialogue_id", "metric_value", "dominant_relation"]],
    df_ft[["dialogue_id", "metric_value"]],
    on="dialogue_id", suffixes=("_baseline", "_finetuned")
)

# Overall
base_vals = merged["metric_value_baseline"].to_numpy()
ft_vals   = merged["metric_value_finetuned"].to_numpy()
diffs     = ft_vals - base_vals

mean_base = float(base_vals.mean())
mean_ft   = float(ft_vals.mean())
mean_diff = float(diffs.mean())

# Bootstrap CI for mean difference
rng = np.random.default_rng(0)
B = 5000
boot_means = rng.choice(diffs, size=(B, len(diffs)), replace=True).mean(axis=1)
ci_low, ci_high = np.percentile(boot_means, [2.5, 97.5])

# Wilcoxon signed-rank
stat, p_wilcox = wilcoxon(ft_vals, base_vals, zero_method="wilcox")

# Chance test: proportion of finetuned dialogues > 0.5
n = len(ft_vals)
n_above = int((ft_vals > 0.5).sum())
p_chance = binom_p_greater(n_above, n, 0.5)

# Print overall table (BART-style)
overall_df = pd.DataFrame([{
    "n_dialogues": n,
    "baseline_mean_acc": mean_base,
    "finetuned_mean_acc": mean_ft,
    "mean_diff": mean_diff,
    "boot_CI_low": float(ci_low),
    "boot_CI_high": float(ci_high),
    "wilcoxon_stat": float(stat),
    "wilcoxon_p": float(p_wilcox),
    "chance_test_p(finetuned>0.5)": float(p_chance),
}])
print(overall_df.to_string(index=False, float_format="%.6f"))

# Relation-wise
rel_rows = []
for rel, group in merged.groupby("dominant_relation"):
    b = group["metric_value_baseline"].to_numpy()
    f = group["metric_value_finetuned"].to_numpy()
    d = f - b

    mean_b = float(b.mean())
    mean_f = float(f.mean())
    mean_d = float(d.mean())

    # Wilcoxon per relation if enough data & non-constant diffs
    if len(group) >= 5 and np.ptp(d) > 0:
        try:
            stat_rel, p_rel = wilcoxon(f, b, zero_method="wilcox")
        except ValueError:
            p_rel = np.nan
    else:
        p_rel = np.nan

    # Chance test per relation
    n_rel = len(group)
    n_above_rel = int((f > 0.5).sum())
    p_chance_rel = binom_p_greater(n_above_rel, n_rel, 0.5) if n_rel > 0 else np.nan

    rel_rows.append({
        "relation_type": rel,
        "n": n_rel,
        "baseline_mean_acc": mean_b,
        "finetuned_mean_acc": mean_f,
        "mean_diff": mean_d,
        "wilcoxon_p(baseline_vs_finetuned)": p_rel,
        "chance_test_p(finetuned>0.5)": p_chance_rel,
    })

rel_df = pd.DataFrame(rel_rows).sort_values("n", ascending=False)
print("\n")
print(rel_df.to_string(index=False, float_format="%.6f"))

# Summary line
print(f"\nSignificance (RoBERTa NSP). The improvement from baseline (acc = {mean_base:.4f}) "
      f"to fine-tuned (acc = {mean_ft:.4f}) was "
      f"{'statistically significant' if p_wilcox < 0.05 else 'not statistically significant'} "
      f"(Wilcoxon signed-rank, p = {p_wilcox:.3g}; 95% CI for mean difference: "
      f"[{ci_low:.3f}, {ci_high:.3f}]). "
      f"Fine-tuned performance was "
      f"{'significantly' if p_chance < 0.05 else 'not significantly'} above chance "
      f"(acc > 0.5; p = {p_chance:.3g}).")

 n_dialogues  baseline_mean_acc  finetuned_mean_acc  mean_diff  boot_CI_low  boot_CI_high  wilcoxon_stat  wilcoxon_p  chance_test_p(finetuned>0.5)
         293           0.510239            0.697099   0.186860     0.154721      0.218999    1086.000000    0.000000                      0.000000


         relation_type  n  baseline_mean_acc  finetuned_mean_acc  mean_diff  wilcoxon_p(baseline_vs_finetuned)  chance_test_p(finetuned>0.5)
  Question_answer_pair 88           0.508523            0.700758   0.192235                           0.000000                      0.000000
               Comment 52           0.500000            0.629808   0.129808                           0.002595                      0.008767
       Acknowledgement 37           0.479730            0.702703   0.222973                           0.000776                      0.000096
           Elaboration 21           0.559524            0.690476   0.130952                           0.020921                      0.013302